# Language-Agnostic Quality Model V2
Updates:
* More features (categories, wikilinks)
* Extract all features from wikitext instead of links tables
* New minimums for "top-quality" to account for new extraction methods
* New model weights based on English, French, and Arabic articles

More details: https://meta.wikimedia.org/wiki/Research:Prioritization_of_Wikipedia_Articles/Language-Agnostic_Quality


## Imports / settings / etc.

In [1]:
import re

import wmfdata

In [2]:
spark = wmfdata.spark.get_session(app_name='pyspark large; quality-model-features-wikitext',
                                  type='yarn-large', # local, yarn-regular, yarn-large
                                  )  

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [3]:
spark.sql("SET hive.exec.dynamic.partition.mode = nonstrict")

DataFrame[key: string, value: string]

In [4]:
print("Wikidata:")
spark.sql('show partitions wmf.wikidata_item_page_link').show(50, False)
print("\nMediawiki:")
spark.sql("show partitions wmf_raw.mediawiki_project_namespace_map").show(50, False)

Wikidata:
+-------------------+
|partition          |
+-------------------+
|snapshot=2021-12-13|
|snapshot=2021-12-20|
|snapshot=2021-12-27|
|snapshot=2022-01-03|
|snapshot=2022-01-10|
|snapshot=2022-01-17|
+-------------------+


Mediawiki:
+------------------------+
|partition               |
+------------------------+
|snapshot=2016-12_private|
|snapshot=2017-07_private|
|snapshot=2021-07        |
|snapshot=2021-08        |
|snapshot=2021-09        |
|snapshot=2021-10        |
|snapshot=2021-11        |
|snapshot=2021-12        |
+------------------------+



## Parameters

In [5]:
mw_snapshot = '2021-12'  # data will be current to this date -- e.g., 2020-07 means data is up to 30 June 2020 (at least)
wd_snapshot = '2022-01-03'
allwikis_tablename = 'isaacj.qual_features_regex_2021_12_wikitext'
qual_preds_tablename = 'isaacj.qual_preds_2021_12'

## Utils

In [6]:
MEDIA_PREFIXES = ['File', 'Image', 'Media']
CAT_PREFIXES = ['Category']

MEDIA_ALIASES = {
    "ab": ["Медиа", "Файл", "Афаил", "Амедиа", "Изображение"],
    "ace": ["Beureukaih", "Gambar", "Alat", "Berkas"],
    "ady": ["Медиа"],
    "af": ["Lêer", "Beeld"],
    "als": ["Medium", "Datei", "Bild"],
    "am": ["ፋይል", "ስዕል"],
    "an": ["Imachen", "Imagen"],
    "ang": ["Ymele", "Biliþ"],
    "ar": ["ميديا", "صورة", "وسائط", "ملف"],
    "arc": ["ܠܦܦܐ", "ܡܝܕܝܐ"],
    "arz": ["ميديا", "صورة", "وسائط", "ملف"],
    "as": ["চিত্ৰ", "चित्र", "চিত্র", "মাধ্যম"],
    "ast": ["Imaxen", "Ficheru", "Imaxe", "Archivu", "Imagen", "Medios"],
    "atj": ["Tipatcimoctakewin", "Natisinahikaniwoc"],
    "av": ["Медиа", "Файл", "Изображение"],
    "ay": ["Medio", "Archivo", "Imagen"],
    "az": ["Mediya", "Şəkil", "Fayl"],
    "azb": ["رسانه", "تصویر", "مدیا", "فایل", "رسانه‌ای"],
    "ba": ["Медиа", "Рәсем", "Файл", "Изображение"],
    "bar": ["Medium", "Datei", "Bild"],
    "bat-smg": ["Vaizdas", "Medėjė", "Abruozdielis"],
    "bcl": ["Medio", "Ladawan"],
    "be": ["Мультымедыя", "Файл", "Выява"],
    "be-x-old": ["Мэдыя", "Файл", "Выява"],
    "bg": ["Медия", "Файл", "Картинка"],
    "bh": ["मीडिया", "चित्र"],
    "bjn": ["Barakas", "Gambar", "Berkas"],
    "bm": ["Média", "Fichier"],
    "bn": ["চিত্র", "মিডিয়া"],
    "bpy": ["ছবি", "মিডিয়া"],
    "br": ["Skeudenn", "Restr"],
    "bs": ["Mediji", "Slika", "Datoteka", "Medija"],
    "bug": ["Gambar", "Berkas"],
    "bxr": ["Файл", "Меди", "Изображение"],
    "ca": ["Fitxer", "Imatge"],
    "cbk-zam": ["Medio", "Archivo", "Imagen"],
    "cdo": ["文件", "媒體", "圖像", "檔案"],
    "ce": ["Хlум", "Медиа", "Сурт", "Файл", "Медйа", "Изображение"],
    "ceb": ["Payl", "Medya", "Imahen"],
    "ch": ["Litratu"],
    "ckb": ["میدیا", "پەڕگە"],
    "co": ["Immagine"],
    "crh": ["Медиа", "Resim", "Файл", "Fayl", "Ресим"],
    "cs": ["Soubor", "Média", "Obrázok"],
    "csb": ["Òbrôzk", "Grafika"],
    "cu": ["Видъ", "Ви́дъ", "Дѣло", "Срѣдьства"],
    "cv": ["Медиа", "Ӳкерчĕк", "Изображение"],
    "cy": ["Delwedd"],
    "da": ["Billede", "Fil"],
    "de": ["Medium", "Datei", "Bild"],
    "din": ["Ciɛl", "Apamduööt"],
    "diq": ["Medya", "Dosya"],
    "dsb": ["Wobraz", "Dataja", "Bild", "Medija"],
    "dty": ["चित्र", "मिडिया"],
    "dv": ["ފައިލު", "މީޑިއާ", "ފައިލް"],
    "el": ["Εικόνα", "Αρχείο", "Μέσο", "Μέσον"],
    "eml": ["Immagine"],
    "eo": ["Dosiero", "Aŭdvidaĵo"],
    "es": ["Medio", "Archivo", "Imagen"],
    "et": ["Pilt", "Fail", "Meedia"],
    "eu": ["Irudi", "Fitxategi"],
    "ext": ["Archivu", "Imagen", "Mediu"],
    "fa": ["رسانه", "تصویر", "مدیا", "پرونده", "رسانه‌ای"],
    "ff": ["Média", "Fichier"],
    "fi": ["Kuva", "Tiedosto"],
    "fiu-vro": ["Pilt", "Meediä"],
    "fo": ["Miðil", "Mynd"],
    "fr": ["Média", "Fichier"],
    "frp": ["Émâge", "Fichiér", "Mèdia"],
    "frr": ["Medium", "Datei", "Bild"],
    "fur": ["Immagine", "Figure"],
    "fy": ["Ofbyld"],
    "ga": ["Íomhá", "Meán"],
    "gag": ["Mediya", "Medya", "Resim", "Dosya", "Dosye"],
    "gan": ["媒体文件", "文件", "文檔", "档案", "媒體", "图像", "圖像", "媒体", "檔案"],
    "gd": ["Faidhle", "Meadhan"],
    "gl": ["Imaxe", "Ficheiro", "Arquivo", "Imagem"],
    "glk": ["رسانه", "تصویر", "پرونده", "فاىل", "رسانه‌ای", "مديا"],
    "gn": ["Medio", "Imagen", "Ta'ãnga"],
    "gom": ["माध्यम", "मिडिया", "फायल"],
    "gor": ["Gambar", "Berkas"],
    "got": ["𐍆𐌴𐌹𐌻𐌰"],
    "gu": ["દ્રશ્ય-શ્રાવ્ય (મિડિયા)", "દ્રશ્ય-શ્રાવ્ય_(મિડિયા)", "ચિત્ર"],
    "gv": ["Coadan", "Meanyn"],
    "hak": ["文件", "媒體", "圖像", "檔案"],
    "haw": ["Kiʻi", "Waihona", "Pāpaho"],
    "he": ["תמונה", "קו", "מדיה", "קובץ"],
    "hi": ["मीडिया", "चित्र"],
    "hif": ["file", "saadhan"],
    "hr": ["Mediji", "DT", "Slika", "F", "Datoteka"],
    "hsb": ["Wobraz", "Dataja", "Bild"],
    "ht": ["Imaj", "Fichye", "Medya"],
    "hu": ["Kép", "Fájl", "Média"],
    "hy": ["Պատկեր", "Մեդիա"],
    "ia": ["Imagine", "Multimedia"],
    "id": ["Gambar", "Berkas"],
    "ig": ["Nká", "Midia", "Usòrò", "Ákwúkwó orünotu", "Ákwúkwó_orünotu"],
    "ii": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "ilo": ["Midia", "Papeles"],
    "inh": ["Медиа", "Файл", "Изображение"],
    "io": ["Imajo", "Arkivo"],
    "is": ["Miðill", "Mynd"],
    "it": ["Immagine"],
    "ja": ["メディア", "ファイル", "画像"],
    "jbo": ["velsku", "datnyvei"],
    "jv": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "ka": ["მედია", "სურათი", "ფაილი"],
    "kaa": ["Swret", "Таспа", "سۋرەت", "Taspa", "Su'wret", "Сурет", "تاسپا"],
    "kab": ["Tugna"],
    "kbd": ["Медиа", "Файл"],
    "kbp": ["Média", "Fichier"],
    "kg": ["Fisye"],
    "kk": ["Swret", "سۋرەت", "Таспа", "Taspa", "Сурет", "تاسپا"],
    "kl": ["Billede", "Fiileq", "Fil"],
    "km": ["ឯកសារ", "រូបភាព", "មេឌា", "មីឌា"],
    "kn": ["ಚಿತ್ರ", "ಮೀಡಿಯ"],
    "ko": ["미디어", "파일", "그림"],
    "koi": ["Медиа", "Файл", "Изображение"],
    "krc": ["Медиа", "Файл", "Изображение"],
    "ks": ["میڈیا", "فَیِل"],
    "ksh": ["Beld", "Meedije", "Medie", "Belld", "Medium", "Datei", "Meedijum", "Bild"],
    "ku": ["میدیا", "پەڕگە", "Medya", "Wêne"],
    "kv": ["Медиа", "Файл", "Изображение"],
    "kw": ["Restren"],
    "ky": ["Медиа", "Файл"],
    "la": ["Imago", "Fasciculus"],
    "lad": ["Dossia", "Medya", "Archivo", "Dosya", "Imagen", "Meddia"],
    "lb": ["Fichier", "Bild"],
    "lbe": ["Медиа", "Сурат", "Изображение"],
    "lez": ["Медиа", "Mediya", "Файл", "Şəkil", "Изображение"],
    "lfn": ["Fix"],
    "li": ["Afbeelding", "Plaetje", "Aafbeilding"],
    "lij": ["Immaggine", "Immagine"],
    "lmo": ["Immagine", "Imàjine", "Archivi"],
    "ln": ["Média", "Fichier"],
    "lo": ["ສື່ອ", "ສື່", "ຮູບ"],
    "lrc": ["رسانه", "تصویر", "رسانه‌ای", "جانیا", "أسگ", "ڤارئسگأر"],
    "lt": ["Vaizdas", "Medija"],
    "ltg": ["Medeja", "Fails"],
    "lv": ["Attēls"],
    "mai": ["मेडिया", "फाइल"],
    "map-bms": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "mdf": ["Медиа", "Няйф", "Изображение"],
    "mg": ["Rakitra", "Sary", "Média"],
    "mhr": ["Медиа", "Файл", "Изображение"],
    "min": ["Gambar", "Berkas"],
    "mk": ["Податотека", "Медија", "Медиум", "Слика"],
    "ml": ["പ്രമാണം", "ചി", "മീഡിയ", "പ്ര", "ചിത്രം"],
    "mn": ["Медиа", "Файл", "Зураг"],
    "mr": ["चित्र", "मिडिया"],
    "mrj": ["Медиа", "Файл", "Изображение"],
    "ms": ["Fail", "Imej"],
    "mt": ["Midja", "Medja", "Stampa"],
    "mwl": ["Multimédia", "Fexeiro", "Ficheiro", "Arquivo", "Imagem"],
    "my": ["ဖိုင်", "မီဒီယာ"],
    "myv": ["Медия", "Артовкс", "Изображение"],
    "mzn": ["رسانه", "تصویر", "مه‌دیا", "مدیا", "پرونده", "رسانه‌ای"],
    "nah": ["Mēdiatl", "Īxiptli", "Imagen"],
    "nap": ["Fiùra", "Immagine"],
    "nds": ["Datei", "Bild"],
    "nds-nl": ["Ofbeelding", "Afbeelding", "Bestaand"],
    "ne": ["मीडिया", "चित्र"],
    "new": ["किपा", "माध्यम"],
    "nl": ["Bestand", "Afbeelding"],
    "nn": ["Fil", "Bilde", "Filpeikar"],
    "no": ["Fil", "Medium", "Bilde"],
    "nov": [],
    "nrm": ["Média", "Fichier"],
    "nso": ["Seswantšho"],
    "nv": ["Eʼelyaaígíí"],
    "oc": ["Imatge", "Fichièr", "Mèdia"],
    "olo": ["Kuva", "Medii", "Failu"],
    "or": ["ମାଧ୍ୟମ", "ଫାଇଲ"],
    "os": ["Ныв", "Медиа", "Файл", "Изображение"],
    "pa": ["ਤਸਵੀਰ", "ਮੀਡੀਆ"],
    "pcd": ["Média", "Fichier"],
    "pdc": ["Medium", "Datei", "Bild", "Feil"],
    "pfl": ["Dadai", "Medium", "Datei", "Bild"],
    "pi": ["मीडिया", "पटिमा"],
    "pl": ["Plik", "Grafika"],
    "pms": ["Figura", "Immagine"],
    "pnb": ["میڈیا", "تصویر", "فائل"],
    "pnt": ["Εικόνα", "Αρχείον", "Εικόναν", "Μέσον"],
    "ps": ["انځور", "رسنۍ", "دوتنه"],
    "pt": ["Multimédia", "Ficheiro", "Arquivo", "Imagem"],
    "qu": ["Midya", "Imagen", "Rikcha"],
    "rm": ["Multimedia", "Datoteca"],
    "rmy": ["Fişier", "Mediya", "Chitro", "Imagine"],
    "ro": ["Fişier", "Imagine", "Fișier"],
    "roa-rup": ["Fişier", "Imagine", "Fișier"],
    "roa-tara": ["Immagine"],
    "ru": ["Медиа", "Файл", "Изображение"],
    "rue": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "rw": ["Dosiye", "Itangazamakuru"],
    "sa": ["चित्रम्", "माध्यमम्", "सञ्चिका", "माध्यम", "चित्रं"],
    "sah": ["Миэдьийэ", "Ойуу", "Билэ", "Изображение"],
    "sat": ["ᱨᱮᱫ", "ᱢᱤᱰᱤᱭᱟ"],
    "sc": ["Immàgini"],
    "scn": ["Immagine", "Mmàggini", "Mèdia"],
    "sd": ["عڪس", "ذريعات", "فائل"],
    "se": ["Fiila"],
    "sg": ["Média", "Fichier"],
    "sh": ["Mediji", "Slika", "Медија", "Datoteka", "Medija", "Слика"],
    "si": ["රූපය", "මාධ්‍යය", "ගොනුව"],
    "sk": ["Súbor", "Obrázok", "Médiá"],
    "sl": ["Slika", "Datoteka"],
    "sq": ["Figura", "Skeda"],
    "sr": ["Датотека", "Medij", "Slika", "Медија", "Datoteka", "Медиј", "Medija", "Слика"],
    "srn": ["Afbeelding", "Gefre"],
    "stq": ["Bielde", "Bild"],
    "su": ["Média", "Gambar"],
    "sv": ["Fil", "Bild"],
    "sw": ["Faili", "Picha"],
    "szl": ["Plik", "Grafika"],
    "ta": ["படிமம்", "ஊடகம்"],
    "tcy": ["ಮಾದ್ಯಮೊ", "ಫೈಲ್"],
    "te": ["ఫైలు", "దస్త్రం", "బొమ్మ", "మీడియా"],
    "tet": ["Imajen", "Arquivo", "Imagem"],
    "tg": ["Акс", "Медиа"],
    "th": ["ไฟล์", "สื่อ", "ภาพ"],
    "ti": ["ፋይል", "ሜድያ"],
    "tk": ["Faýl"],
    "tl": ["Midya", "Talaksan"],
    "tpi": ["Fail"],
    "tr": ["Medya", "Resim", "Dosya", "Ortam"],
    "tt": ["Медиа", "Рәсем", "Файл", "Räsem", "Изображение"],
    "ty": ["Média", "Fichier"],
    "tyv": ["Медиа", "Файл", "Изображение"],
    "udm": ["Медиа", "Файл", "Суред", "Изображение"],
    "ug": ["ۋاسىتە", "ھۆججەت"],
    "uk": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "ur": ["میڈیا", "تصویر", "وسیط", "زریعہ", "فائل", "ملف"],
    "uz": ["Mediya", "Tasvir", "Fayl"],
    "vec": ["Immagine", "Imàjine", "Mèdia"],
    "vep": ["Pilt", "Fail"],
    "vi": ["Phương_tiện", "Tập_tin", "Hình", "Tập tin", "Phương tiện"],
    "vls": ["Afbeelding", "Ofbeeldienge"],
    "vo": ["Ragiv", "Magod", "Nünamakanäd"],
    "wa": ["Imådje"],
    "war": ["Medya", "Fayl", "Paypay"],
    "wo": ["Xibaarukaay", "Dencukaay"],
    "wuu": ["文件", "档案", "图像", "媒体"],
    "xal": ["Аһар", "Боомг", "Изображение", "Зург"],
    "xmf": ["მედია", "სურათი", "ფაილი"],
    "yi": ["מעדיע", "תמונה", "טעקע", "בילד"],
    "yo": ["Fáìlì", "Amóhùnmáwòrán", "Àwòrán"],
    "za": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "zea": ["Afbeelding", "Plaetje"],
    "zh": ["媒体文件", "F", "文件", "媒體", "档案", "图像", "圖像", "媒体", "檔案"],
    "zh-classical": ["文件", "媒體", "圖像", "檔案"],
    "zh-min-nan": ["tóng-àn", "文件", "媒體", "Mûi-thé", "圖像", "檔案"],
    "zh-yue": ["檔", "档", "文件", "图", "媒體", "圖", "档案", "图像", "圖像", "媒体", "檔案"],
}

CAT_ALIASES = {
    "ab": ["Категория", "Акатегориа"],
    "ace": ["Kawan", "Kategori"],
    "af": ["Kategorie"],
    "ak": ["Nkyekyem"],
    "als": ["Kategorie"],
    "am": ["መደብ"],
    "an": ["Categoría"],
    "ang": ["Flocc"],
    "ar": ["تصنيف"],
    "arc": ["ܣܕܪܐ"],
    "arz": ["تصنيف"],
    "as": ["CAT", "শ্ৰেণী", "श्रेणी", "শ্রেণী"],
    "ast": ["Categoría"],
    "atj": ["Tipanictawin"],
    "av": ["Категория"],
    "ay": ["Categoría"],
    "az": ["Kateqoriya"],
    "azb": ["بؤلمه"],
    "ba": ["Төркөм", "Категория"],
    "bar": ["Kategorie"],
    "bat-smg": ["Kategorija", "Kateguorėjė"],
    "bcl": ["Kategorya"],
    "be": ["Катэгорыя"],
    "be-x-old": ["Катэгорыя"],
    "bg": ["Категория"],
    "bh": ["श्रेणी"],
    "bjn": ["Tumbung", "Kategori"],
    "bm": ["Catégorie"],
    "bn": ["বিষয়শ্রেণী", "വിഭാഗം"],
    "bpy": ["থাক"],
    "br": ["Rummad"],
    "bs": ["Kategorija"],
    "bug": ["Kategori"],
    "bxr": ["Категори", "Категория"],
    "ca": ["Categoria"],
    "cbk-zam": ["Categoría"],
    "cdo": ["分類"],
    "ce": ["Категори", "Тоба", "Кадегар"],
    "ceb": ["Kategoriya"],
    "ch": ["Katigoria"],
    "ckb": ["پ", "پۆل"],
    "co": ["Categoria"],
    "crh": ["Категория", "Kategoriya"],
    "cs": ["Kategorie"],
    "csb": ["Kategòrëjô"],
    "cu": ["Катигорї", "Категория", "Катигорїꙗ"],
    "cv": ["Категори"],
    "cy": ["Categori"],
    "da": ["Kategori"],
    "de": ["Kategorie"],
    "din": ["Bekätakthook"],
    "diq": ["Kategoriye", "Kategori"],
    "dsb": ["Kategorija"],
    "dty": ["श्रेणी"],
    "dv": ["ޤިސްމު"],
    "el": ["Κατηγορία"],
    "eml": ["Categoria"],
    "eo": ["Kategorio"],
    "es": ["CAT", "Categoría"],
    "et": ["Kategooria"],
    "eu": ["Kategoria"],
    "ext": ["Categoría", "Categoria"],
    "fa": ["رده"],
    "ff": ["Catégorie"],
    "fi": ["Luokka"],
    "fiu-vro": ["Katõgooria"],
    "fo": ["Bólkur"],
    "fr": ["Catégorie"],
    "frp": ["Catègorie"],
    "frr": ["Kategorie"],
    "fur": ["Categorie"],
    "fy": ["Kategory"],
    "ga": ["Rang", "Catagóir"],
    "gag": ["Kategori", "Kategoriya"],
    "gan": ["分類", "分类"],
    "gd": ["Roinn-seòrsa"],
    "gl": ["Categoría"],
    "glk": ["جرگه", "رده"],
    "gn": ["Ñemohenda"],
    "gom": ["वर्ग", "श्रेणी"],
    "gor": ["Dalala"],
    "got": ["𐌷𐌰𐌽𐍃𐌰"],
    "gu": ["શ્રેણી", "CAT", "શ્રે"],
    "gv": ["Ronney"],
    "hak": ["分類"],
    "haw": ["Māhele"],
    "he": ["קטגוריה", "קט"],
    "hi": ["श्र", "श्रेणी"],
    "hif": ["vibhag"],
    "hr": ["CT", "KT", "Kategorija"],
    "hsb": ["Kategorija"],
    "ht": ["Kategori"],
    "hu": ["Kategória"],
    "hy": ["Կատեգորիա"],
    "ia": ["Categoria"],
    "id": ["Kategori"],
    "ie": ["Categorie"],
    "ig": ["Ébéonọr", "Òtù"],
    "ii": ["分类"],
    "ilo": ["Kategoria"],
    "inh": ["ОагӀат"],
    "io": ["Kategorio"],
    "is": ["Flokkur"],
    "it": ["CAT", "Categoria"],
    "ja": ["カテゴリ"],
    "jbo": ["klesi"],
    "jv": ["Kategori"],
    "ka": ["კატეგორია"],
    "kaa": ["Sanat", "Kategoriya", "Санат", "سانات"],
    "kab": ["Taggayt"],
    "kbd": ["Категория", "Категориэ"],
    "kbp": ["Catégorie"],
    "kg": ["Kalasi"],
    "kk": ["Sanat", "Санат", "سانات"],
    "kl": ["Sumut_atassuseq", "Kategori", "Sumut atassuseq"],
    "km": ["ចំនាត់ថ្នាក់ក្រុម", "ចំណាត់ក្រុម", "ចំណាត់ថ្នាក់ក្រុម"],
    "kn": ["ವರ್ಗ"],
    "ko": ["분류"],
    "koi": ["Категория"],
    "krc": ["Категория"],
    "ks": ["زٲژ"],
    "ksh": ["Saachjropp", "Saachjrop", "Katejori", "Kategorie", "Saachjrupp", "Kattejori", "Sachjrop"],
    "ku": ["Kategorî", "پۆل"],
    "kv": ["Категория"],
    "kw": ["Class", "Klass"],
    "ky": ["Категория"],
    "la": ["Categoria"],
    "lad": ["Kateggoría", "Katēggoría", "Categoría"],
    "lb": ["Kategorie"],
    "lbe": ["Категория"],
    "lez": ["Категория"],
    "lfn": ["Categoria"],
    "li": ["Categorie", "Kategorie"],
    "lij": ["Categorîa", "Categoria"],
    "lmo": ["Categuria", "Categoria"],
    "ln": ["Catégorie"],
    "lo": ["ໝວດ"],
    "lrc": ["دأسە"],
    "lt": ["Kategorija"],
    "ltg": ["Kategoreja"],
    "lv": ["Kategorija"],
    "mai": ["CA", "श्रेणी"],
    "map-bms": ["Kategori"],
    "mdf": ["Категорие", "Категория"],
    "mg": ["Sokajy", "Catégorie"],
    "mhr": ["Категория", "Категорий"],
    "min": ["Kategori"],
    "mk": ["Категорија"],
    "ml": ["വിഭാഗം", "വി", "വർഗ്ഗം", "വ"],
    "mn": ["Ангилал"],
    "mr": ["वर्ग"],
    "mrj": ["Категори", "Категория"],
    "ms": ["Kategori"],
    "mt": ["Kategorija"],
    "mwl": ["Catadorie", "Categoria"],
    "my": ["ကဏ္ဍ"],
    "myv": ["Категория"],
    "mzn": ["رج", "رده"],
    "nah": ["Neneuhcāyōtl", "Categoría"],
    "nap": ["Categurìa", "Categoria"],
    "nds": ["Kategorie"],
    "nds-nl": ["Categorie", "Kattegerie", "Kategorie"],
    "ne": ["श्रेणी"],
    "new": ["पुचः"],
    "nl": ["Categorie"],
    "nn": ["Kategori"],
    "no": ["Kategori"],
    "nrm": ["Catégorie"],
    "nso": ["Setensele"],
    "nv": ["Tʼááłáhági_átʼéego", "Tʼááłáhági átʼéego"],
    "oc": ["Categoria"],
    "olo": ["Kategourii"],
    "or": ["ବିଭାଗ", "ଶ୍ରେଣୀ"],
    "os": ["Категори"],
    "pa": ["ਸ਼੍ਰੇਣੀ"],
    "pcd": ["Catégorie"],
    "pdc": ["Abdeeling", "Kategorie"],
    "pfl": ["Kadegorie", "Sachgrubb", "Kategorie"],
    "pi": ["विभाग"],
    "pl": ["Kategoria"],
    "pms": ["Categorìa"],
    "pnb": ["گٹھ"],
    "pnt": ["Κατηγορίαν"],
    "ps": ["وېشنيزه"],
    "pt": ["Categoria"],
    "qu": ["Katiguriya"],
    "rm": ["Categoria"],
    "rmy": ["Shopni"],
    "ro": ["Categorie"],
    "roa-rup": ["Categorie"],
    "roa-tara": ["Categoria"],
    "ru": ["Категория", "К"],
    "rue": ["Категория", "Катеґорія"],
    "rw": ["Ikiciro"],
    "sa": ["वर्गः"],
    "sah": ["Категория"],
    "sat": ["ᱛᱷᱚᱠ"],
    "sc": ["Categoria"],
    "scn": ["Catigurìa"],
    "sd": ["زمرو"],
    "se": ["Kategoriija"],
    "sg": ["Catégorie"],
    "sh": ["Kategorija", "Категорија"],
    "si": ["ප්‍රවර්ගය"],
    "sk": ["Kategória"],
    "sl": ["Kategorija"],
    "sq": ["Kategoria", "Kategori"],
    "sr": ["Kategorija", "Категорија"],
    "srn": ["Categorie", "Guru"],
    "stq": ["Kategorie"],
    "su": ["Kategori"],
    "sv": ["Kategori"],
    "sw": ["Jamii"],
    "szl": ["Kategoryjo", "Kategoria"],
    "ta": ["பகுப்பு"],
    "tcy": ["ವರ್ಗೊ"],
    "te": ["వర్గం"],
    "tet": ["Kategoría", "Kategoria"],
    "tg": ["Гурӯҳ"],
    "th": ["หมวดหมู่"],
    "ti": ["መደብ"],
    "tk": ["Kategoriýa"],
    "tl": ["Kategorya", "Kaurian"],
    "tpi": ["Grup"],
    "tr": ["Kategori", "KAT"],
    "tt": ["Төркем", "Törkem", "Категория"],
    "ty": ["Catégorie"],
    "tyv": ["Аңгылал", "Категория"],
    "udm": ["Категория"],
    "ug": ["تۈر"],
    "uk": ["Категория", "Категорія"],
    "ur": ["زمرہ"],
    "uz": ["Turkum", "Kategoriya"],
    "vec": ["Categoria"],
    "vep": ["Kategorii"],
    "vi": ["Thể_loại", "Thể loại"],
    "vls": ["Categorie"],
    "vo": ["Klad"],
    "wa": ["Categoreye"],
    "war": ["Kaarangay"],
    "wo": ["Wàll", "Catégorie"],
    "wuu": ["分类"],
    "xal": ["Янз", "Әәшл"],
    "xmf": ["კატეგორია"],
    "yi": ["קאטעגאריע", "קאַטעגאָריע"],
    "yo": ["Ẹ̀ka"],
    "za": ["分类"],
    "zea": ["Categorie"],
    "zh": ["分类", "分類", "CAT"],
    "zh-classical": ["分類", "CAT"],
    "zh-min-nan": ["分類", "Lūi-pia̍t"],
    "zh-yue": ["分类", "分類", "类", "類"],
}

In [7]:
def getArticleFeaturesRegex(wikitext, wiki_db='enwiki', level=100):
    """Gather counts of article components directly from wikitext.
    
    Pros:
    * Much faster than mwparserfromhell (10x speed-up in testing) -- e.g., 200 µs vs. 2 ms for medium-sized article
    * Can be easily extended to catch edge-cases -- e.g., images added via infoboxes/galleries that lack bracket syntax
    
    Cons/Issues:
    * Misses intra-nested links:
        * e.g. [[File:Image.jpg|Image with a [[caption]]]] only catches the File and not the [[caption]]
        * Could be extended by also regexing each link found, which should catch almost all
    * Misses references added via templates w/o ref tags -- e.g., shortened-footnote templates.
    * Misses media added via templates / gallery tags that lack brackets
    """
    try:
        lang = wiki_db.replace('wiki', '')
        cat_prefixes = CAT_PREFIXES + CAT_ALIASES.get(lang, [])
        med_prefixes = MEDIA_PREFIXES + MEDIA_ALIASES.get(lang, [])
        page_length = len(wikitext)
        ref_singleton = re.compile(r'<ref(\s[^/>]*)?/>', re.M | re.I)
        ref_tag = re.compile(r'<ref(\s[^/>]*)?>[\s\S]*?</ref>', re.M | re.I)
        num_ref_tags =  len(ref_singleton.findall(wikitext)) + len(ref_tag.findall(wikitext))
        links = [m for m in re.findall('(?<=\[\[)(.*?)(?=\]\])', wikitext, flags=re.DOTALL)]
        num_categories = len([1 for l in links if l.split(':')[0] in cat_prefixes])
        num_media = len([1 for l in links if l.split(':')[0] in med_prefixes])
        num_wikilinks = len(links) - num_categories - num_media
        num_headings = len([1 for l in re.findall('(={2,})(.*?)(={2,})', wikitext) if len(l[0]) <= level])
        return [(page_length, num_ref_tags, num_wikilinks, num_categories, num_media, num_headings)]
    except Exception:
        return None
    
# To be used as: SELECT INLINE(getArticleFeatures(wikitext)) FROM ...
# The array part is incidental -- no method for inline pure structs...
spark.udf.register('getArticleFeatures', getArticleFeaturesRegex,
                   'ARRAY<STRUCT<page_length:INT, num_refs:INT, num_wikilinks:INT, num_categories:INT, num_media:INT, num_headings:INT>>')


<function __main__.getArticleFeaturesRegex(wikitext, wiki_db='enwiki', level=100)>

## Create Table for Data

In [66]:
do_execute = True
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {allwikis_tablename} (
        page_id          INT     COMMENT 'Article page ID',
        page_length      INT     COMMENT 'Number of bytes in article wikitext',
        num_refs         INT     COMMENT 'Number of references in article',
        num_wikilinks    INT     COMMENT 'Number of wikilinks in article',
        num_categories   INT     COMMENT 'Number of categories in article',
        num_media        INT     COMMENT 'Number of unique images in article',
        num_headings     INT     COMMENT 'Number of Level-2 and Level-3 headings in article'
    )
    PARTITIONED BY (
        wiki_db          STRING  COMMENT 'Wiki dbname -- e.g., enwiki for English Wikipedia'
    )
    """

if do_execute:
    spark.sql(create_table_query)

## Generate feature data for all wikis

In [ ]:
"""
The quality model requires the following attributes:
* Quantity:
  * # bytes (page length)
  * # links
  * # of headers (just levels 2 + 3)
  * # of images
* Accessibility
  * bytes / header
* Reliability
  * # of references
* Annotations
  * # of categories

Not included but considered:
* Wikidata item completeness
* alt-text coverage
* issue templates
* # of assessments / WikiProjects


Explanation of CTEs:
* 
"""

print_for_hive = False
do_execute = True

query = f"""
WITH wikipedia_projects AS (
    SELECT DISTINCT
      dbname
    FROM wmf_raw.mediawiki_project_namespace_map
    WHERE
      snapshot = '{mw_snapshot}'
      AND hostname LIKE '%.wikipedia%'
),
pages AS (
    SELECT
      wiki_db,
      page_id
    FROM wmf_raw.mediawiki_page p
    INNER JOIN wikipedia_projects wp
      ON (p.wiki_db = wp.dbname)
    WHERE
      snapshot = '{mw_snapshot}'
      AND page_namespace = 0
      AND NOT page_is_redirect
),
features AS (
     SELECT
      wt.wiki_db,
      wt.page_id,
      INLINE(getArticleFeatures(revision_text, wt.wiki_db, 3))
    FROM wmf.mediawiki_wikitext_current wt
    INNER JOIN pages p
      ON (wt.page_id = p.page_id
          AND wt.wiki_db = p.wiki_db)
    WHERE
      snapshot = '{mw_snapshot}'
      AND page_namespace = 0
)
INSERT OVERWRITE TABLE {allwikis_tablename}
SELECT
  p.page_id,
  COALESCE(page_length, 0) AS page_length,
  COALESCE(num_refs, 0) AS num_refs,
  COALESCE(num_wikilinks, 0) AS num_wikilinks,
  COALESCE(num_categories, 0) AS num_categories,
  COALESCE(num_media, 0) AS num_media,
  COALESCE(num_headings, 0) AS num_headings,
  p.wiki_db AS wiki_db
FROM pages p
LEFT JOIN features f
  ON (p.wiki_db = f.wiki_db
      AND p.page_id = f.page_id)
"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    result = spark.sql(query)


WITH wikipedia_projects AS (
    SELECT DISTINCT
      dbname
    FROM wmf_raw.mediawiki_project_namespace_map
    WHERE
      snapshot = '2021-12'
      AND hostname LIKE '%.wikipedia%'
),
pages AS (
    SELECT
      wiki_db,
      page_id
    FROM wmf_raw.mediawiki_page p
    INNER JOIN wikipedia_projects wp
      ON (p.wiki_db = wp.dbname)
    WHERE
      snapshot = '2021-12'
      AND page_namespace = 0
      AND NOT page_is_redirect
),
features AS (
     SELECT
      wt.wiki_db,
      wt.page_id,
      INLINE(getArticleFeatures(revision_text, wt.wiki_db, 3))
    FROM wmf.mediawiki_wikitext_current wt
    INNER JOIN pages p
      ON (wt.page_id = p.page_id
          AND wt.wiki_db = p.wiki_db)
    WHERE
      snapshot = '2021-12'
      AND page_namespace = 0
)
INSERT OVERWRITE TABLE isaacj.qual_features_regex_2021_12_wikitext
SELECT
  p.page_id,
  COALESCE(page_length, 0) AS page_length,
  COALESCE(num_refs, 0) AS num_refs,
  COALESCE(num_wikilinks, 0) AS num_wikilinks,
  COALESCE

In [6]:
spark.sql(f'select * from {allwikis_tablename} where wiki_db = "enwiki" limit 10').show(500, False)


+-------+-----------+--------+-------------+-------------+--------------+---------+------------+-------+
|page_id|page_length|num_refs|num_wikilinks|num_templates|num_categories|num_media|num_headings|wiki_db|
+-------+-----------+--------+-------------+-------------+--------------+---------+------------+-------+
|1793   |41064      |26      |771          |40           |2             |0        |15          |enwiki |
|3386   |93333      |127     |570          |295          |20            |38       |32          |enwiki |
|3836   |9627       |7       |33           |7            |9             |0        |9           |enwiki |
|4243   |97156      |84      |408          |109          |6             |21       |30          |enwiki |
|4431   |75452      |217     |182          |127          |7             |4        |19          |enwiki |
|5726   |8799       |3       |43           |7            |3             |4        |5           |enwiki |
|5906   |144592     |180     |500          |386        

## Apply Model

In [7]:
def predictQuality(sqrt_length, num_media, num_headings_per, num_refs_per, num_cats, sqrt_links_per):
    """Predict quality of article."""
    
    coef_len = 0.395
    coef_med = 0.114
    coef_ref = 0.181
    coef_hea = 0.123
    coef_lin = 0.115
    coef_cat = 0.070
    try:
        pred = ((coef_len * sqrt_length) + 
                (coef_med * num_media) + 
                (coef_ref * num_refs_per) +
                (coef_hea * num_headings_per) + 
                (coef_lin * sqrt_links_per) +
                (coef_cat * num_cats))
        return pred
    except Exception:
        return None
        
spark.udf.register('predictQuality', predictQuality, 'Float')

<function __main__.predictQuality(sqrt_length, num_media, num_headings_per, num_refs_per, num_cats, sqrt_links_per)>

In [17]:
do_execute = True
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {qual_preds_tablename} (
        page_id          INT     COMMENT 'Article page ID',
        item_id          STRING  COMMENT 'Associated Wikidata QID',
        pred_qual        FLOAT   COMMENT 'Predicted quality score [0-1]; 0 = no content; 1 = highest quality'
    )
    PARTITIONED BY (
        wiki_db          STRING  COMMENT 'Wiki dbname -- e.g., enwiki for English Wikipedia'
    )
    """

if do_execute:
    spark.sql(create_table_query)

In [19]:
print_for_hive = False
do_execute = True

# 95% percentile captures top quality but filters out extreme outliers
qual_pctile = 0.95

# Determined empirically -- these are straightforward because they are raw counts
MIN_MEDIA = 2
MIN_CATS = 5

# Determined empirically and equivalent to 10000 characters which is exceeded even in Chinese Wikipedia (12526 characters in 95th percentile)
# Remember it's a single transformation: sqrt(# bytes in page)
MIN_SQRTLENGTH = 100 

# Determined empirically -- mostly penalizes bot-driven wikis that have many articles with lede paragraphs so 0 sections
# Remember it's a single transformation: # headings / sqrt(# bytes in page)
# Equivalent to 1 section @ 100 characters; 2 sections at 400 characters; 3 sections at 900 characters etc.
MIN_HEADINGS_PER = 0.1

# Determined empirically but wikis it affects are mixed bags (some bot-driven wikis have very heavily-referenced short articles)
# Many wikis are closer to 0.25 - 0.3 range
# Remember it's a single transformation: # references / sqrt(# bytes in page)
# Equivalent to 1 ref @ 45 characters; 2 refs at 180 characters; 3 refs at 400 characters etc.
# Or ~2 references per section taking the min_headings_per logic
MIN_REFS_PER = 0.15

# Determined empirically -- almost all wikis are over 0.1 with CJK getting close to 0.2
# Remember it's a double transformation: sqrt(# wikilinks) / sqrt(# bytes in page)
# Equivalent to 1 link per 100 characters, which is ~1 link per sentence.
MIN_SQRTLINKS_PER = 0.1

query = f"""
WITH wikipedia_projects AS (
    SELECT DISTINCT
      dbname,
      SUBSTR(hostname, 1, LENGTH(hostname)-4) AS project
    FROM wmf_raw.mediawiki_project_namespace_map
    WHERE
      snapshot = '{mw_snapshot}'
      AND hostname LIKE '%wikipedia%'
),
all_pages AS (
    SELECT
      wiki_db,
      page_id
    FROM wmf_raw.mediawiki_page p
    INNER JOIN wikipedia_projects wp
      ON (p.wiki_db = wp.dbname)
    WHERE
      snapshot = '{mw_snapshot}'
      AND page_namespace = 0
      AND NOT page_is_redirect
),
wikidata_ids AS (
    SELECT
      wiki_db,
      page_id,
      item_id
    FROM wmf.wikidata_item_page_link wd
    INNER JOIN wikipedia_projects wp
      ON (wd.wiki_db = wp.dbname)
    WHERE
      snapshot = '{wd_snapshot}'
      AND page_namespace = 0
),
max_vals AS (
    SELECT
      wiki_db,
      LEAST({MIN_SQRTLENGTH}, PERCENTILE_APPROX(SQRT(page_length), {qual_pctile})) AS max_length,
      LEAST({MIN_MEDIA}, PERCENTILE(num_media, {qual_pctile})) AS max_media,
      LEAST({MIN_HEADINGS_PER}, PERCENTILE_APPROX(num_headings / SQRT(page_length), {qual_pctile})) AS max_headings,
      LEAST({MIN_REFS_PER}, PERCENTILE_APPROX(num_refs / SQRT(page_length), {qual_pctile})) AS max_refs,
      LEAST({MIN_CATS}, PERCENTILE(num_categories, {qual_pctile})) AS max_cats,
      LEAST({MIN_SQRTLINKS_PER}, PERCENTILE_APPROX(SQRT(num_wikilinks)  / SQRT(page_length), {qual_pctile})) AS max_links
    FROM {allwikis_tablename} qf
    INNER JOIN wikipedia_projects wp
      ON (qf.wiki_db = wp.dbname)
    WHERE
      page_length IS NOT NULL
      AND num_media IS NOT NULL
      AND num_headings IS NOT NULL
      AND num_refs IS NOT NULL
      AND num_categories IS NOT NULL
      AND num_wikilinks IS NOT NULL
    GROUP BY
      wiki_db
),
qual_features_trimmed AS (
    SELECT
      page_id,
      qf.wiki_db,
      COALESCE(LEAST(SQRT(page_length), max_length), 0) / max_length AS len_x,
      COALESCE(LEAST(num_media, max_media), 0) / max_media AS media_x,
      COALESCE(LEAST(num_headings / SQRT(page_length), max_headings), 0) / max_headings AS headings_x,
      COALESCE(LEAST(num_refs / SQRT(page_length), max_refs), 0) / max_refs AS refs_x,
      COALESCE(LEAST(num_categories, max_cats), 0) / max_cats AS cats_x,
      COALESCE(LEAST(SQRT(num_wikilinks) / SQRT(page_length), max_links), 0) / max_links AS links_x
    FROM {allwikis_tablename} qf
    INNER JOIN max_vals mv
      ON (qf.wiki_db = mv.wiki_db)
),
qual_predictions AS (
    SELECT
      wiki_db,
      page_id,
      COALESCE(predictQuality(len_x, media_x, headings_x, refs_x, cats_x, links_x), 0) AS pred_qual
    FROM qual_features_trimmed
)
INSERT OVERWRITE TABLE {qual_preds_tablename}
SELECT
  ap.page_id,
  wd.item_id,
  COALESCE(pred_qual, 0),
  ap.wiki_db
FROM all_pages ap
LEFT JOIN qual_predictions qp
  ON (ap.wiki_db = qp.wiki_db
      AND ap.page_id = qp.page_id)
LEFT JOIN wikidata_ids wd
  ON (ap.wiki_db = wd.wiki_db
      AND ap.page_id = wd.page_id)
"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    result = spark.sql(query)


WITH wikipedia_projects AS (
    SELECT DISTINCT
      dbname,
      SUBSTR(hostname, 1, LENGTH(hostname)-4) AS project
    FROM wmf_raw.mediawiki_project_namespace_map
    WHERE
      snapshot = '2021-12'
      AND hostname LIKE '%wikipedia%'
),
all_pages AS (
    SELECT
      wiki_db,
      page_id
    FROM wmf_raw.mediawiki_page p
    INNER JOIN wikipedia_projects wp
      ON (p.wiki_db = wp.dbname)
    WHERE
      snapshot = '2021-12'
      AND page_namespace = 0
      AND NOT page_is_redirect
),
wikidata_ids AS (
    SELECT
      wiki_db,
      page_id,
      item_id
    FROM wmf.wikidata_item_page_link wd
    INNER JOIN wikipedia_projects wp
      ON (wd.wiki_db = wp.dbname)
    WHERE
      snapshot = '2022-01-03'
      AND page_namespace = 0
),
max_vals AS (
    SELECT
      wiki_db,
      LEAST(100, PERCENTILE_APPROX(SQRT(page_length), 0.95)) AS max_length,
      LEAST(2, PERCENTILE(num_media, 0.95)) AS max_media,
      LEAST(0.1, PERCENTILE_APPROX(num_headings / SQRT(page_len

In [20]:
spark.sql(f'SELECT * FROM {qual_preds_tablename} LIMIT 50').show(100, False)

+-------+---------+----------+-------+
|page_id|item_id  |pred_qual |wiki_db|
+-------+---------+----------+-------+
|5867   |Q444     |0.1913985 |sawiki |
|6610   |Q32767   |0.40396875|sawiki |
|7163   |Q23743   |0.40400082|sawiki |
|7948   |Q6813    |0.4039678 |sawiki |
|11988  |Q15419   |0.4067006 |sawiki |
|12208  |Q185790  |0.6383765 |sawiki |
|13489  |Q16037350|0.5994925 |sawiki |
|13660  |Q16042887|0.3104578 |sawiki |
|14237  |Q2046270 |0.42218298|sawiki |
|14551  |Q7601355 |0.40711966|sawiki |
|17852  |Q31830010|0.5917713 |sawiki |
|21567  |Q525166  |0.49075964|sawiki |
|24754  |Q711996  |0.2526131 |sawiki |
|25255  |Q797114  |0.4587993 |sawiki |
|25262  |Q16042877|0.38737532|sawiki |
|25621  |Q798839  |0.41196966|sawiki |
|26842  |Q912899  |0.55779374|sawiki |
|30424  |Q16274099|0.39437312|sawiki |
|30901  |Q2389445 |0.25766295|sawiki |
|36428  |Q3022    |0.3190889 |sawiki |
|67844  |Q97264426|0.7131208 |sawiki |
|68031  |Q97264482|0.6485031 |sawiki |
|69010  |Q97264467|0.7778

## Regex vs. mwparserfromhell
Evaluation of speed and check for differences in results. Custom regexes are at least an order of magnitude faster than mwparserfromhell. There are a few differences in results that arise from nested links but nothing major.

In [18]:
import mwparserfromhell

def getArticleFeaturesMWP(wikitext, wiki_db='enwiki'):
    try:
        lang = wiki_db.replace('wiki', '')
        cat_prefixes = CAT_PREFIXES + CAT_ALIASES.get(lang, [])
        med_prefixes = MEDIA_PREFIXES + MEDIA_ALIASES.get(lang, [])
        wt = mwparserfromhell.parse(wikitext)
        page_length = len(wikitext)
        num_ref_tags = len([t.tag for t in wt.filter_tags() if t.tag == 'ref'])
        links = [l.title for l in wt.filter_wikilinks()]
        num_categories = len([1 for l in links if l.split(':')[0] in cat_prefixes])
        num_media = len([1 for l in links if l.split(':')[0] in med_prefixes])
        num_wikilinks = len(links) - num_categories - num_media
        num_headings = len(wt.filter_headings())
        # num_sftn_templates = len([t.name for t in wt.filter_templates() if t.name.lower() in sfn_templates])
        return [(page_length, num_ref_tags, num_wikilinks, num_templates, num_categories, num_media, num_headings)] # + num_sftn_templates
    except Exception:
        return None

In [8]:
wt = """{{Short description|Austrian painter}}
'''Karl Josef Aigen''' (8 October 1684 – 22 October 1762) was a landscape painter, born at [[Olomouc]].
==Life==
[[File:Carl Aigen Fischmarkt.jpg|thumb|''Fischmarkt'' by Karl Aigen]]
Aigen was born in Olomouc on 8 October 1685, the son of a goldsmith.
He was a pupil of the Olomouc painter Dominik Maier. He lived in [[Vienna]] from about 1720, where he was professor of painting at the [[Academy of Fine Arts Vienna|Academy]] from 1751 until his death. His work consists of landscapes with figures, genre paintings and altarpieces. His style shows the influence of artists from France and the Low Countries.<ref name=belv>{{cite web|title=Karl Josef Aigen|url=http://digital.belvedere.at/emuseum/view/people/asitem/items$0040null:13/0?t:state:flow=85945fe1-2502-4798-a332-087cc860da49|publisher=Belvedere Wien|accessdate=27 March 2014}}</ref>
He died at Vienna on 21 October 1762.<ref name=belv/>
===Works===
The [[Österreichische Galerie Belvedere|Gallery of the Belvedere]] in Vienna has two works by him, both scenes with figures.<ref>{{Bryan (3rd edition)|title=Aigen, Karl |volume=1}}</ref>
==References==
{{reflist}}
==External links==
{{cite web|title=Works in the [[Belveddere Gallery]]|url=http://digital.belvedere.at/emuseum/view/objects/asimages/search$0040?t:state:flow=3a74c35b-1547-43a3-a2b8-5bc257d26adb|publisher=Digitales Belvedere}}
{{commons category}}
{{Authority control}}
{{Use dmy dates|date=April 2017}}
{{DEFAULTSORT:Aigen, Karl}}
[[Category:1684 births]]
[[Category:1762 deaths]]
[[Category:17th-century Austrian painters]]
[[Category:18th-century Austrian painters]]
[[Category:Academy of Fine Arts Vienna alumni]]
[[Category:Academy of Fine Arts Vienna faculty]]
[[Category:Austrian male painters]]
[[Category:Moravian-German people]]
[[Category:People from the Margraviate of Moravia]]
[[Category:Artists from Olomouc]]
{{Austria-painter-stub}}
"""

long_wt = """{{short description|Political philosophy and movement}}
{{other uses}}
{{redirect2|Anarchist|Anarchists|other uses|Anarchist (disambiguation)}}
{{pp-semi-indef}}
{{good article}}
{{use British English|date=August 2021}}
{{use dmy dates|date=August 2021}}
{{anarchism sidebar}}
{{basic forms of government}}
'''Anarchism''' is a [[political philosophy]] and [[Political movement|movement]] that is sceptical of [[authority]] and rejects all involuntary, coercive forms of [[Social hierarchy|hierarchy]]. Anarchism calls for the abolition of the [[State (polity)|state]], which it holds to be unnecessary, undesirable, and harmful. As a historically [[left-wing]] movement, placed on the farthest left of the [[political spectrum]], it is usually described alongside [[libertarian Marxism]] as the [[libertarian]] wing ([[libertarian socialism]]) of the [[socialist movement]], and has a strong historical association with [[anti-capitalism]] and [[socialism]].

[[Humans]] lived in [[society|societies]] without formal hierarchies long before the establishment of formal states, [[realm]]s, or [[empire]]s. With the rise of organised hierarchical bodies, [[scepticism]] toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the [[Age of Enlightenment|Enlightenment]]. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in workers' struggles for [[emancipation]]. Various [[anarchist schools of thought]] formed during this period. Anarchists have taken part in several revolutions, most notably in the [[Paris Commune]], the [[Russian Civil War]] and the [[Spanish Civil War]], whose end marked the end of the [[classical era of anarchism]]. In the last decades of the 20th and into the 21st century, the anarchist movement has been resurgent once more.

Anarchism employs a [[diversity of tactics]] in order to meet its ideal ends which can be broadly separated into revolutionary and evolutionary tactics; there is significant overlap between the two, which are merely descriptive. Revolutionary tactics aim to bring down authority and state, having taken a violent turn in the past, while evolutionary tactics aim to prefigure what an anarchist society would be like. Anarchist thought, criticism, and [[Praxis (process)|praxis]] have played a part in diverse areas of human society. Criticism of anarchism include claims that it is internally inconsistent, violent, or utopian.
{{toc limit|3}}

== Etymology, terminology, and definition ==
{{main|Definition of anarchism and libertarianism}}
{{see also|Glossary of anarchism}}
[[File:WilhelmWeitling.jpg|thumb|[[Wilhelm Weitling]] is an example of a writer who added to anarchist theory without using the exact term.{{sfn|Carlson|1972|pp=22–23}}]]
The etymological origin of ''anarchism'' is from the Ancient Greek ''anarkhia'', meaning "without a ruler", composed of the prefix ''an-'' ("without") and the word ''arkhos'' ("leader" or "ruler"). The suffix ''[[-ism]]'' denotes the ideological current that favours [[anarchy]].{{sfnm|1a1=Bates|1y=2017|1p=128|2a1=Long|2y=2013|2p=217}} ''Anarchism'' appears in English from 1642 as ''anarchisme'' and ''anarchy'' from 1539; early English usages emphasised a sense of disorder.{{sfnm|1a1=Merriam-Webster|1y=2019|1loc="Anarchism"|2a1=''Oxford English Dictionary''|2y=2005|2loc="Anarchism"|3a1=Sylvan|3y=2007|3p=260}} Various factions within the [[French Revolution]] labelled their opponents as ''anarchists'', although few such accused shared many views with later anarchists. Many revolutionaries of the 19th century such as [[William Godwin]] (1756–1836) and [[Wilhelm Weitling]] (1808–1871) would contribute to the anarchist doctrines of the next generation but did not use ''anarchist'' or ''anarchism'' in describing themselves or their beliefs.{{sfn|Joll|1964|pp=27–37}}

The first political philosopher to call himself an ''anarchist'' ({{lang-fr|anarchiste}}) was [[Pierre-Joseph Proudhon]] (1809–1865), marking the formal birth of anarchism in the mid-19th century. Since the 1890s and beginning in France,{{sfn|Nettlau|1996|p=162}} ''[[libertarianism]]'' has often been used as a synonym for anarchism{{sfn|Guérin|1970|loc="The Basic Ideas of Anarchism"}} and its use as a synonym is still common outside the United States.{{sfnm|1a1=Ward|1y=2004|1p=62|2a1=Goodway|2y=2006|2p=4|3a1=Skirda|3y=2002|3p=183|4a1=Fernández|4y=2009|4p=9}} Some usages of ''libertarianism'' refer to [[individualistic]] [[free-market]] philosophy only, and [[free-market anarchism]] in particular is termed ''libertarian anarchism''.{{sfn|Morris|2002|p=61}}

While the term ''libertarian'' has been largely synonymous with anarchism,{{sfnm|1a1=Marshall|1y=1992|1p=641|2a1=Cohn|2y=2009|2p=6}} its meaning has more recently diluted with wider adoption from ideologically disparate groups,{{sfn|Marshall|1992|p=641}} including both the [[New Left]] and [[libertarian Marxists]], who do not associate themselves with [[authoritarian socialists]] or a [[vanguard party]], and extreme [[cultural liberals]], who are primarily concerned with [[civil liberties]].{{sfn|Marshall|1992|p=641}} Additionally, some anarchists use ''[[libertarian socialist]]''{{sfnm|1a1=Marshall|1y=1992|1p=641|2a1=Chomsky|2y=2005|2p=123|3a1=Cohn|3y=2009|3p=6|4a1=Levy|4a2=Adams|4y=2018|4p=104}} to avoid anarchism's negative connotations and emphasise its connections with [[socialism]].{{sfn|Marshall|1992|p=641}} ''Anarchism'' is broadly used to describe the [[anti-authoritarian]] wing of the [[socialist movement]].{{sfn|Levy|Adams|2018|p=104}}{{refn|In ''Anarchism: From Theory to Practice'' (1970),{{sfn|Guérin|1970|p=12}} anarchist historian [[Daniel Guérin]] described it as a synonym for [[libertarian socialism]], and wrote that anarchism "is really a synonym for socialism. The anarchist is primarily a socialist whose aim is to abolish the exploitation of man by man. Anarchism is only one of the streams of socialist thought, that stream whose main components are concern for liberty and haste to abolish the State."{{sfn|Arvidsson|2017}} In his many works on anarchism, such as ''Chomsky on Anarchism'' (2005),{{sfn|Chomsky|2005|p=123}} historian [[Noam Chomsky]] describes anarchism, alongside [[libertarian Marxism]], as the [[libertarian]] wing of [[socialism]].{{sfn|Otero|1994|p=617}}|group=nb}} Anarchism is contrasted to socialist forms which are state-oriented or from above.{{sfn|Osgood|1889|p=1}} Scholars of anarchism generally highlight anarchism's socialist credentials{{sfn|Newman|2005|p=15}} and criticise attempts at creating dichotomies between the two.{{sfn|Morris|2015|p=64}} Some scholars describe anarchism as having many influences from [[liberalism]],{{sfn|Marshall|1992|p=641}} and being both liberals and socialists but more so,{{sfn|Walter|2002|p=44}} while most scholars reject [[anarcho-capitalism]] as a misunderstanding of anarchist principles.{{sfnm|1a1=Marshall|1y=1992|1pp=564–565|2a1=Jennings|2y=1993|2p=143|3a1=Gay|3a2=Gay|3y=1999|3p=15|4a1=Morris|4y=2008|4p=13|5a1=Johnson|5y=2008|5p=169|6a1=Franks|6y=2013|6pp=393–394}}{{refn|[[Herbert L. Osgood]] claimed that anarchism is "the extreme antithesis" of [[authoritarian communism]] and [[state socialism]].{{sfn|Osgood|1889|p=1}} [[Peter Marshall (author)|Peter Marshall]] states that "[i]n general anarchism is closer to socialism than liberalism. ... Anarchism finds itself largely in the socialist camp, but it also has outriders in liberalism. It cannot be reduced to socialism, and is best seen as a separate and distinctive doctrine."{{sfn|Marshall|1992|p=641}} According to [[Jeremy Jennings]], "[i]t is hard not to conclude that these ideas", referring to [[anarcho-capitalism]], "are described as anarchist only on the basis of a misunderstanding of what anarchism is." Jennings adds that "anarchism does not stand for the untrammelled freedom of the individual (as the 'anarcho-capitalists' appear to believe) but, as we have already seen, for the extension of individuality and community."{{sfn|Jennings|1999|p=147}} [[Nicolas Walter]] wrote that "anarchism does derive from liberalism and socialism both historically and ideologically. ... In a sense, anarchists always remain liberals and socialists, and whenever they reject what is good in either they betray anarchism itself. ... We are liberals but more so, and socialists but more so."{{sfn|Walter|2002|p=44}} Michael Newman includes anarchism as one of many [[socialist traditions]], especially the more socialist-aligned tradition following Proudhon and [[Mikhail Bakunin]].{{sfn|Newman|2005|p=15}} [[Brian Morris (anthropologist)|Brian Morris]] argues that it is "conceptually and historically misleading" to "create a dichotomy between socialism and anarchism."{{sfn|Morris|2015|p=64}}|group=nb}}

While [[opposition to the state]] is central to anarchist thought, defining ''anarchism'' is not an easy task for scholars, as there is a lot of discussion among scholars and anarchists on the matter, and various currents perceive anarchism slightly differently.{{sfn|Long|2013|p=217}}{{refn|One common definition adopted by anarchists is that anarchism is a cluster of political philosophies opposing [[authority]] and [[hierarchical organisation]], including [[Anarchism and capitalism|capitalism]], [[Anarchism and nationalism|nationalism]], the [[State (polity)|state]], and all associated [[institution]]s, in the conduct of all [[human relations]] in favour of a society based on [[decentralisation]], [[freedom]], and [[voluntary association]]. Scholars highlight that this definition has the same shortcomings as the definition based on anti-authoritarianism (''[[a posteriori]]'' conclusion), anti-statism (anarchism is much more than that),{{sfnm|1a1=McLaughlin|1y=2007|1p=166|2a1=Jun|2y=2009|2p=507|3a1=Franks|3y=2013|3pp=386–388}} and etymology (negation of rulers).{{sfnm|1a1=McLaughlin|1y=2007|1pp=25–29|2a1=Long|2y=2013|2pp=217}}|group=nb}} Major definitional elements include the will for a non-coercive society, the rejection of the state apparatus, the belief that human nature allows humans to exist in or progress toward such a non-coercive society, and a suggestion on how to act to pursue the ideal of anarchy.{{sfn|McLaughlin|2007|pp=25–26}}

== History ==
{{main|History of anarchism}}

=== Pre-modern era ===
[[File:Paolo Monti - Servizio fotografico (Napoli, 1969) - BEIC 6353768.jpg|thumb|upright=.7|[[Zeno of Citium]] (c. 334 – c. 262 BC), whose ''[[Republic (Zeno)|Republic]]'' inspired [[Peter Kropotkin]]{{sfn|Marshall|1993|p=70}}]]
Before the establishment of towns and cities, an established authority did not exist. It was after the creation of institutions of authority that anarchistic ideas espoused as a reaction.{{sfn|Graham|2005|pp=xi–xiv}} The most notable precursors to anarchism in the ancient world were in China and Greece. In China, [[philosophical anarchism]] (the discussion on the legitimacy of the state) was delineated by [[Taoist]] philosophers [[Zhuang Zhou]] and [[Laozi]].{{sfnm|1a1=Coutinho|1y=2016|2a1=Marshall|2y=1993|2p=54}} Alongside [[Stoicism]], Taoism has been said to have had "significant anticipations" of anarchism.{{Sfn|Sylvan|2007|p=257}}
 
Anarchic attitudes were also articulated by tragedians and philosophers in Greece. [[Aeschylus]] and [[Sophocles]] used the myth of [[Antigone]] to illustrate the conflict between rules set by the state and personal [[autonomy]]. [[Socrates]] questioned Athenian authorities constantly and insisted on the right of individual freedom of conscience. [[Cynicism (philosophy)|Cynics]] dismissed human law (''[[Nomos (sociology)|nomos]]'') and associated authorities while trying to live according to nature (''[[physis]]''). [[Stoics]] were supportive of a society based on unofficial and friendly relations among its citizens without the presence of a state.{{sfn|Marshall|1993|pp=4, 66–73}}

In [[Middle Ages|medieval Europe]], there was no anarchistic activity except some ascetic religious movements. These, and other Muslim movements, later gave birth to [[religious anarchism]]. In the [[Sasanian Empire]], [[Mazdak]] called for an [[egalitarian]] society and the [[abolition of monarchy]], only to be soon executed by Emperor [[Kavad I]].{{sfn|Marshall|1993|p=86}}

In [[Basra]], religious sects preached against the state.<ref>{{cite journal|last=Crone|first=Patricia|title=Ninth-Century Muslim Anarchists.|journal=[[Past & Present (journal)|Past & Present]]|volume=167|year=2000|pages=3, 21–25<!--3–28-->|url=https://www.hs.ias.edu/files/Crone_Articles/Crone_Ninth_Century_Muslim_Anarchists.pdf|doi=10.1093/past/167.1.3}}</ref> In Europe, various sects developed anti-state and libertarian tendencies.{{sfn|Nettlau|1996|p=8}} Renewed interest in antiquity during the [[Renaissance]] and in private judgment during the [[Reformation]] restored elements of anti-authoritarian secularism, particularly in France.{{sfn|Marshall|1993|p=108}} [[Age of Enlightenment|Enlightenment]] challenges to intellectual authority (secular and religious) and the [[List of revolutions and rebellions|revolutions of the 1790s and 1848]] all spurred the ideological development of what became the era of classical anarchism.{{sfn|Levy|Adams|2018|p=307}}

=== Modern era ===
During the [[French Revolution]], partisan groups such as the [[Enragés]] and the {{lang|fr|[[sans-culottes]]}} saw a turning point in the fermentation of anti-state and federalist sentiments.{{sfn|Marshall|1993|p=4}} The first anarchist currents developed throughout the 18th century as [[William Godwin]] espoused [[philosophical anarchism]] in England, morally delegitimising the state, [[Max Stirner]]'s thinking paved the way to [[Individualist anarchism|individualism]] and [[Pierre-Joseph Proudhon]]'s theory of [[Mutualism (economic theory)|mutualism]] found fertile soil in France.{{sfn|Marshall|1993|pp=4–5}} By the late 1870s, various anarchist schools of thought had become well-defined and a wave of then unprecedented [[globalisation]] occurred from 1880 to 1914.{{sfn|Levy|2011|pp=10–15}} This era of [[classical anarchism]] lasted until the end of the [[Spanish Civil War]] and is considered the golden age of anarchism.{{sfn|Marshall|1993|pp=4–5}}

[[File:Bakunin.png|thumb|upright|Mikhail Bakunin opposed the Marxist aim of [[dictatorship of the proletariat]] and allied himself with the federalists in the First International before his expulsion by the Marxists.]]
Drawing from mutualism, [[Mikhail Bakunin]] founded [[collectivist anarchism]] and entered the [[International Workingmen's Association]], a class worker union later known as the First International that formed in 1864 to unite diverse revolutionary currents. The International became a significant political force, with [[Karl Marx]] being a leading figure and a member of its General Council. Bakunin's faction (the [[Jura Federation]]) and Proudhon's followers (the mutualists) opposed [[state socialism]], advocating political [[abstentionism]] and small property holdings.{{sfnm|1a1=Dodson|1y=2002|1p=312|2a1=Thomas|2y=1985|2p=187|3a1=Chaliand|3a2=Blin|3y=2007|3p=116}} After bitter disputes, the Bakuninists were expelled from the International by the [[Marxists]] at the [[1872 Hague Congress]].{{sfnm|1a1=Graham|1y=2019|1pp=334–336|2a1=Marshall|2y=1993|2p=24}} Anarchists were treated similarly in the [[Second International]], being ultimately expelled in 1896.{{sfn|Levy|2011|p=12}} Bakunin famously predicted that if revolutionaries gained power by Marx's terms, they would end up the new tyrants of workers. In response to their expulsion from the First International, anarchists formed the [[St. Imier International]]. Under the influence of [[Peter Kropotkin]], a Russian philosopher and scientist, [[anarcho-communism]] overlapped with collectivism.{{sfn|Marshall|1993|p=5}} Anarcho-communists, who drew inspiration from the 1871 [[Paris Commune]], advocated for free federation and for the distribution of goods according to one's needs.{{sfn|Graham|2005|p=xii}}

At the turn of the century, anarchism had spread all over the world.{{sfn|Moya|2015|p=327}} It was a notable feature of the international syndicalism movement.{{sfn|Levy|2011|p=16}} In China, small groups of students imported the humanistic pro-science version of anarcho-communism.{{sfn|Marshall|1993|pp=519–521}} Tokyo was a hotspot for rebellious youth from countries of the far east, travelling to the Japanese capital to study.{{sfnm|1a1=Dirlik|1y=1991|1p=133|2a1=Ramnath|2y=2019|2pp=681–682}} In Latin America, [[Anarchism in Argentina|Argentina]] was a stronghold for [[anarcho-syndicalism]], where it became the most prominent left-wing ideology.{{sfnm|1a1=Levy|1y=2011|1p=23|2a1=Laursen|2y=2019|2p=157|3a1=Marshall|3y=1993|3pp=504–508}} During this time, a minority of anarchists adopted tactics of revolutionary [[political violence]]. This strategy became known as [[propaganda of the deed]].{{sfn|Marshall|1993|pp=633–636}} The dismemberment of the French socialist movement into many groups and the execution and exile of many [[Communards]] to penal colonies following the suppression of the Paris Commune favoured individualist political expression and acts.{{sfn|Anderson|2004}} Even though many anarchists distanced themselves from these terrorist acts, infamy came upon the movement and attempts were made to exclude them from American immigration, including the [[Immigration Act of 1903]], also called the Anarchist Exclusion Act.{{sfnm|1a1=Marshall|1y=1993|1pp=633–636|2a1=Lutz|2a2=Ulmschneider|2y=2019|2p=46}} [[Illegalism]] was another strategy which some anarchists adopted during this period.{{sfn|Bantman|2019|p=374}}

[[File:Makhno group.jpg|thumb|left|Nestor Makhno seen with members of the anarchist [[Revolutionary Insurrectionary Army of Ukraine]]]]
Despite concerns, anarchists enthusiastically participated in the [[Russian Revolution]] in opposition to the [[White movement]]; however, they met harsh suppression after the [[Bolshevik government]] was stabilised. Several anarchists from Petrograd and Moscow fled to Ukraine,{{sfn|Avrich|2006|p=204}} notably leading to the [[Kronstadt rebellion]] and [[Nestor Makhno]]'s struggle in the [[Free Territory]]. With the anarchists being crushed in Russia, two new antithetical currents emerged, namely [[platformism]] and [[synthesis anarchism]]. The former sought to create a coherent group that would push for revolution while the latter were against anything that would resemble a political party. Seeing the victories of the [[Bolsheviks]] in the [[October Revolution]] and the resulting [[Russian Civil War]], many workers and activists turned to [[communist parties]] which grew at the expense of anarchism and other socialist movements. In France and the United States, members of major syndicalist movements such as the [[General Confederation of Labour (France)|General Confederation of Labour]] and the [[Industrial Workers of the World]] left their organisations and joined the [[Communist International]].{{sfn|Nomad|1966|p=88}}

In the Spanish Civil War of 1936, anarchists and syndicalists ([[Confederación Nacional del Trabajo|CNT]] and [[Federación Anarquista Ibérica|FAI]]) once again allied themselves with various currents of leftists. A long tradition of [[Spanish anarchism]] led to anarchists playing a pivotal role in the war. In response to the army rebellion, an anarchist-inspired movement of peasants and workers, supported by armed militias, took control of Barcelona and of large areas of rural Spain, where they [[collectivised]] the land.{{sfn|Bolloten|1984|p=1107}} The [[Soviet Union]] provided some limited assistance at the beginning of the war, but the result was a bitter fight among communists and anarchists at a series of events named [[May Days]] as [[Joseph Stalin]] tried to seize control of the [[Republican faction (Spanish Civil War)|Republicans]].{{sfn|Marshall|1993|pp=xi, 466}}

=== Post-war era ===
[[File:Rojava Sewing Cooperative.jpg|thumb|Rojava's support efforts for workers to form cooperatives is exemplified in this sewing cooperative.]]
At the end of [[World War II]], the anarchist movement was severely weakened.{{sfn|Marshall|1993|p=xi}} The 1960s witnessed a revival of anarchism, likely caused by a perceived failure of [[Marxism–Leninism]] and tensions built by the [[Cold War]].{{sfn|Marshall|1993|p=539}} During this time, anarchism found a presence in other movements critical towards both capitalism and the state such as the [[Anti-nuclear movement|anti-nuclear]], [[Environmental movement|environmental]], and [[peace movement]]s, the [[counterculture of the 1960s]], and the [[New Left]].{{sfn|Marshall|1993|pp=xi, 539}} It also saw a transition from its previous revolutionary nature to provocative [[anti-capitalist reform]]ism.{{sfn|Levy|2011|pp=5|p=}} Anarchism became associated with [[punk subculture]] as exemplified by bands such as [[Crass]] and the [[Sex Pistols]].{{sfn|Marshall|1993|pp=493–494}} The established [[feminist]] tendencies of [[anarcha-feminism]] returned with vigour during the [[second wave of feminism]].{{sfn|Marshall|1993|pp=556–557}} [[Black anarchism]] began to take form at this time and influenced anarchism's move from a [[Eurocentric]] demographic.{{sfn|Williams|2015|p=680}} This coincided with its failure to gain traction in Northern Europe and its unprecedented height in Latin America.{{sfn|Harmon|2011|p=70}}

Around the turn of the 21st century, anarchism grew in popularity and influence within [[anti-capitalist]], [[anti-war]] and [[anti-globalisation]] movements.{{sfn|Rupert|2006|p=66}} Anarchists became known for their involvement in protests against the [[World Trade Organization]] (WTO), the [[Group of Eight]] and the [[World Economic Forum]]. During the protests, ''ad hoc'' leaderless anonymous cadres known as [[black bloc]]s engaged in [[riot]]ing, [[property destruction]] and violent confrontations with the [[police]]. Other organisational tactics pioneered in this time include [[affinity group]]s, [[security culture]] and the use of decentralised technologies such as the Internet. A significant event of this period was the confrontations at the [[1999 Seattle WTO conference]].{{sfn|Rupert|2006|p=66}} Anarchist ideas have been influential in the development of the [[Zapatista Army of National Liberation|Zapatistas]] in Mexico and the Democratic Federation of Northern Syria, more commonly known as [[Rojava]], a ''de facto'' [[Permanent autonomous zone|autonomous region]] in northern Syria.{{sfn|Ramnath|2019|p=691}}

== Thought ==
{{main|Anarchist schools of thought}}
Anarchist schools of thought have been generally grouped into two main historical traditions, [[social anarchism]] and [[individualist anarchism]], owing to their different origins, values and evolution.{{sfnm|1a1=McLean|1a2=McMillan|1y=2003|1loc="Anarchism"|2a1=Ostergaard|2y=2003|2p=14|2loc="Anarchism"}} The individualist current emphasises [[negative liberty]] in opposing restraints upon the free individual, while the social current emphasises [[positive liberty]] in aiming to achieve the free potential of society through equality and [[social ownership]].{{sfn|Harrison|Boyd|2003|p=251}} In a chronological sense, anarchism can be segmented by the classical currents of the late 19th century and the post-classical currents ([[anarcha-feminism]], [[green anarchism]], and [[post-anarchism]]) developed thereafter.{{sfn|Levy|Adams|2018|p=9}}

Beyond the specific factions of anarchist movements which constitute political anarchism lies [[philosophical anarchism]] which holds that the state lacks moral legitimacy, without necessarily accepting the imperative of revolution to eliminate it.{{sfn|Egoumenides|2014|p=2}} A component especially of individualist anarchism,{{sfnm|1a1=Ostergaard|1y=2003|1p=12|2a1=Gabardi|2y=1986|2pp=300–302}} philosophical anarchism may tolerate the existence of a [[minimal state]] but claims that citizens have no [[moral obligation]] to obey government when it conflicts with individual autonomy.{{sfn|Klosko|2005|p=4}} Anarchism pays significant attention to moral arguments since [[ethics]] have a central role in anarchist philosophy.{{sfn|Franks|2019|p=549}} Anarchism's emphasis on [[anti-capitalism]], [[egalitarianism]], and for the extension of community and individuality sets it apart from [[anarcho-capitalism]] and other types of [[economic libertarian]]ism.{{sfnm|1a1=Marshall|1y=1992|1pp=564–565|2a1=Jennings|2y=1993|2p=143|3a1=Gay|3a2=Gay|3y=1999|3p=15|4a1=Morris|4y=2008|4p=13|5a1=Johnson|5y=2008|5p=169|6a1=Franks|6y=2013|6pp=393–394}}

Anarchism is usually placed on the [[far-left]] of the [[political spectrum]].{{sfnm|1a1=Brooks|1y=1994|1p=xi|2a1=Kahn|2y=2000|3a1=Moynihan|3y=2007}} Much of its [[Anarchist economics|economics]] and [[Anarchist law|legal philosophy]] reflect [[anti-authoritarian]], [[anti-statist]], [[libertarian]], and [[Radical politics|radical]] interpretations of [[left-wing]] and [[socialist]] politics{{sfn|Guérin|1970|p=12}} such as [[Collectivist anarchism|collectivism]], [[Anarcho-communism|communism]], [[Individualist anarchism|individualism]], [[Mutualism (economic theory)|mutualism]], and [[Anarcho-syndicalism|syndicalism]], among other [[libertarian socialist]] economic theories.{{sfn|Guérin|1970|p=35|loc="Critique of authoritarian socialism"}} As anarchism does not offer a fixed body of doctrine from a single particular worldview,{{sfn|Marshall|1993|pp=14–17}} many anarchist types and traditions exist and varieties of anarchy diverge widely.{{sfn|Sylvan|2007|p=262}} One reaction against [[sectarianism]] within the anarchist milieu was [[anarchism without adjectives]], a call for toleration and unity among anarchists first adopted by [[Fernando Tarrida del Mármol]] in 1889 in response to the bitter debates of anarchist theory at the time.{{sfn|Avrich|1996|p=6}} Belief in political [[nihilism]] has been espoused by anarchists.{{sfn|Walter|2002|p=52}} Despite separation, the various anarchist schools of thought are not seen as distinct entities but rather as tendencies that intermingle and are connected through a set of uniform principles such as individual and local autonomy, mutual aid, network organisation, communal democracy, justified authority and decentralisation.{{sfnm|1a1=Marshall|1y=1993|1p=1–6|2a1=Angelbeck|2a2=Grier|2y=2012|2p=551}}

=== Classical ===
[[File:Portrait of Pierre Joseph Proudhon 1865.jpg|thumb|upright|Pierre-Joseph Proudhon is the primary proponent of mutualism and influenced many future individualist anarchist and social anarchist thinkers.{{sfn|Wilbur|2019|p=216–218}}]]
Inceptive currents among classical anarchist currents were [[Mutualism (economic theory)|mutualism]] and [[Individualist anarchism|individualism]]. They were followed by the major currents of [[social anarchism]] ([[Collectivist anarchism|collectivist]], [[Anarcho-communism|communist]] and [[Anarcho-syndicalism|syndicalist]]). They differ on organisational and economic aspects of their ideal society.{{sfn|Levy|Adams|2018|p=2}}

Mutualism is an 18th-century economic theory that was developed into anarchist theory by [[Pierre-Joseph Proudhon]]. Its aims include [[Reciprocity (cultural anthropology)|reciprocity]], [[Free association (Marxism and anarchism)|free association]], voluntary [[contract]], [[federation]] and [[monetary reform]] of both credit and currency that would be regulated by a bank of the people.{{sfn|Wilbur|2019|pp=213–218}} Mutualism has been retrospectively characterised as ideologically situated between individualist and collectivist forms of anarchism.{{sfnm|1a1=Avrich|1y=1996|1p=6|2a1=Miller|2y=1991|2p=11}} In ''[[What Is Property?]]'' (1840), Proudhon first characterised his goal as a "third form of society, the synthesis of communism and property."{{sfn|Pierson|2013|p=187}} Collectivist anarchism is a [[revolutionary socialist]] form of anarchism{{sfn|Morris|1993|p=76}} commonly associated with [[Mikhail Bakunin]].{{sfn|Shannon|2019|p=101}} Collectivist anarchists advocate [[collective ownership]] of the [[means of production]] which is theorised to be achieved through violent revolution{{sfn|Avrich|1996|pp=3–4}} and that workers be paid according to time worked, rather than goods being distributed according to need as in communism. Collectivist anarchism arose alongside [[Marxism]] but rejected the [[dictatorship of the proletariat]] despite the stated Marxist goal of a collectivist [[stateless society]].{{sfnm|1a1=Heywood|1y=2017|1pp=146–147|2a1=Bakunin|2y=1990}}

[[Anarcho-communism]] is a theory of anarchism that advocates a [[communist society]] with [[common ownership]] of the means of production,{{sfn|Mayne|1999|p=131}} [[direct democracy]] and a [[Horizontalidad|horizontal]] network of [[voluntary association]]s, [[workers' council]]s and [[worker cooperative]]s, with production and consumption based on the guiding principle "[[From each according to his ability, to each according to his need]]."{{sfnm|1a1=Marshall|1y=1993|1p=327|2a1=Turcato|2y=2019|2pp=237–323}} Anarcho-communism developed from radical socialist currents after the [[French Revolution]]{{sfn|Graham|2005}} but was first formulated as such in the Italian section of the [[First International]].{{sfn|Pernicone|2009|pp=111–113}} It was later expanded upon in the theoretical work of [[Peter Kropotkin]],{{sfn|Turcato|2019|pp=239–244}} whose specific style would go onto become the dominating view of anarchists by the late 19th century.{{sfn|Levy|2011|pp=6|p=}} Anarcho-syndicalism is a branch of anarchism that views [[labour syndicate]]s as a potential force for revolutionary social change, replacing capitalism and the state with a new society democratically self-managed by workers. The basic principles of anarcho-syndicalism are [[direct action]], workers' [[solidarity]] and [[workers' self-management]].{{sfn|Van der Walt|2019|p=249}}

Individualist anarchism is a set of several traditions of thought within the anarchist movement that emphasise the [[individual]] and their [[Will (philosophy)|will]] over any kinds of external determinants.{{sfn|Ryley|2019|p=225}} Early influences on individualist forms of anarchism include [[William Godwin]], [[Max Stirner]], and [[Henry David Thoreau]]. Through many countries, individualist anarchism attracted a small yet diverse following of Bohemian artists and intellectuals{{sfn|Marshall|1993|p=440}} as well as young anarchist outlaws in what became known as [[illegalism]] and [[individual reclamation]].{{sfnm|1a1=Imrie|1y=1994|2a1=Parry|2y=1987|2p=15}}

=== Post-classical and contemporary ===
{{main|Contemporary anarchism}}
[[File:Jarach and Zerzan.JPG|thumb|Lawrence Jarach (left) and [[John Zerzan]] (right) are two prominent contemporary anarchist authors, with Zerzan being a prominent voice within [[anarcho-primitivism]] and Jarach a notable advocate of [[post-left anarchy]].]]
Anarchist principles undergird contemporary radical social movements of the left. Interest in the anarchist movement developed alongside momentum in the [[anti-globalisation movement]],{{sfn|Evren|2011|p=1}} whose leading activist networks were anarchist in orientation.{{sfn|Evren|2011|p=2}} As the movement shaped 21st century radicalism, wider embrace of anarchist principles signaled a revival of interest.{{sfn|Evren|2011|p=2}} Anarchism has continued to generate many philosophies and movements, at times eclectic, drawing upon various sources and [[syncretic|combining disparate concepts]] to create new philosophical approaches.{{sfn|Williams|2007|p=303}} The [[anti-capitalist]] tradition of classical anarchism has remained prominent within contemporary currents.{{sfn|Williams|2018|p=4}}

Contemporary news coverage which emphasizes [[black bloc]] demonstrations has reinforced anarchism's historical association with chaos and violence. Its publicity has also led more scholars in fields such as [[anthropology]] and [[history]] to engage with the anarchist movement, although contemporary anarchism favours actions over academic theory.{{sfnm|1a1=Williams|1y=2010|1p=110|2a1=Evren|2y=2011|2p=1|3a1=Angelbeck|3a2=Grier|3y=2012|3p=549}} Various anarchist groups, tendencies, and schools of thought exist today, making it difficult to describe the contemporary anarchist movement.{{sfn|Franks|2013|pp=385–386}} While theorists and activists have established "relatively stable constellations of anarchist principles", there is no consensus on which principles are core and commentators describe multiple ''anarchisms'', rather than a singular ''anarchism'', in which common principles are shared between schools of anarchism while each group prioritizes those principles differently. Gender equality can be a common principle, although it ranks as a higher priority to [[anarcha-feminists]] than [[anarcho-communists]].{{sfn|Franks|2013|p=386}}

Anarchists are generally committed against coercive authority in all forms, namely "all centralized and hierarchical forms of government (e.g., monarchy, representative democracy, state socialism, etc.), economic class systems (e.g., capitalism, Bolshevism, feudalism, slavery, etc.), autocratic religions (e.g., fundamentalist Islam, Roman Catholicism, etc.), patriarchy, heterosexism, white supremacy, and imperialism."{{sfn|Jun|2009|pp=507–508}} Anarchist schools disagree on the methods by which these forms should be opposed.{{sfn|Jun|2009|p=507}} The principle of [[equal liberty]] is closer to anarchist political ethics in that it transcends both the liberal and socialist traditions. This entails that liberty and equality cannot be implemented within the state, resulting in the questioning of all forms of domination and hierarchy.{{sfn|Egoumenides|2014|p=91}}

== Tactics ==
Anarchists' tactics take various forms but in general serve two major goals, namely to first oppose [[the Establishment]] and secondly to promote anarchist ethics and reflect an anarchist vision of society, illustrating the unity of means and ends.{{sfn|Williams|2019|pp=107–108}} A broad categorisation can be made between aims to destroy oppressive states and institutions by revolutionary means on one hand and aims to change society through evolutionary means on the other.{{sfn|Williams|2018|pp=4–5}} Evolutionary tactics embrace [[nonviolence]], reject violence and take a gradual approach to anarchist aims, although there is significant overlap between the two.{{sfn|Kinna|2019|p=125}}

Anarchist tactics have shifted during the course of the last century. Anarchists during the early 20th century focused more on strikes and militancy while contemporary anarchists use a broader array of approaches.{{sfn|Williams|2019|p=112}}

=== Classical era tactics ===
[[File:McKinleyAssassination.jpg|thumb|left|The relationship between [[anarchism and violence]] is a controversial subject among anarchists as shown by anarchist [[Leon Czolgosz]] [[Assassination of William McKinley|assassinating]] [[William McKinley]].]]
During the classical era, anarchists had a militant tendency. Not only did they confront state armed forces, as in Spain and Ukraine, but some of them also employed [[terrorism]] as [[propaganda of the deed]]. Assassination attempts were carried out against heads of state, some of which were successful. Anarchists also took part in [[revolution]]s.{{sfn|Williams|2019|pp=112–113}} Many anarchists, especially the [[Galleanists]], believed that these attempts would be the impetus for a revolution against capitalism and the state.{{sfn|Norris|2020|pp=7-8}} Many of these attacks were done by individual assailants and the majority took place in the late 1870s, the early 1880s and the 1890s, with some still occurring in the early 1900s.{{sfnm|1a1=Levy|1y=2011|1p=13|2a1=Nesser|2y=2012|2p=62}} Their decrease in prevalence was the result of further [[judicial]] power and targeting and cataloging by state institutions.{{sfn|Harmon|2011|p=55}}

Anarchist perspectives towards violence have always been controversial.{{sfn|Carter|1978|p=320}} [[Anarcho-pacifists]] advocate for non-violence means to achieve their stateless, nonviolent ends.{{sfn|Fiala|2017|loc=section 3.1}} Other anarchist groups advocate direct action, a tactic which can include acts of [[sabotage]] or terrorism. This attitude was quite prominent a century ago when seeing the state as a [[tyrant]] and some anarchists believing that they had every right to oppose its [[oppression]] by any means possible.{{sfn|Kinna|2019|pp=116–117}} [[Emma Goldman]] and [[Errico Malatesta]], who were proponents of limited use of violence, stated that violence is merely a reaction to state violence as a [[necessary evil]].{{sfn|Carter|1978|pp=320–325}}

Anarchists took an active role in [[strike action]]s, although they tended to be antipathetic to formal [[syndicalism]], seeing it as [[reformist]]. They saw it as a part of the movement which sought to overthrow the [[State (polity)|state]] and [[capitalism]].{{sfn|Williams|2019|p=113}} Anarchists also reinforced their propaganda within the arts, some of whom practiced [[naturism]] and [[nudism]]. Those anarchists also built communities which were based on [[friendship]] and were involved in the [[news media]].{{sfn|Williams|2019|p=114}}

=== Revolutionary tactics ===
[[File:Black bloc at RNC running.jpg|thumb|Black bloc protesters parading anarcho-communism imagery such as the motto "[[No War but the Class War]]"]]
In the current era, Italian anarchist [[Alfredo Bonanno]], a proponent of [[insurrectionary anarchism]], has reinstated the debate on violence by rejecting the nonviolence tactic adopted since the late 19th century by Kropotkin and other prominent anarchists afterwards. Both Bonanno and the French group [[The Invisible Committee]] advocate for small, informal affiliation groups, where each member is responsible for their own actions but works together to bring down oppression utilizing sabotage and other violent means against state, capitalism, and other enemies. Members of The Invisible Committee were arrested in 2008 on various charges, terrorism included.{{sfn|Kinna|2019|pp=134–135}}

Overall, contemporary anarchists are much less violent and militant than their ideological ancestors. They mostly engage in confronting the police during demonstrations and riots, especially in countries such as [[Anarchism in Canada|Canada]], [[Anarchism in Greece|Greece]], and [[Anarchism in Mexico|Mexico]]. Militant [[black bloc]] protest groups are known for clashing with the police;{{sfn|Williams|2019|p=115}} however, anarchists not only clash with state operators, they also engage in the struggle against fascists and racists, taking [[anti-fascist]] action and mobilizing to prevent hate rallies from happening.{{sfn|Williams|2019|p=117}}

=== Evolutionary tactics ===
Anarchists commonly employ [[direct action]]. This can take the form of disrupting and protesting against unjust [[hierarchy]], or the form of self-managing their lives through the creation of counter-institutions such as communes and non-hierarchical collectives.{{sfn|Williams|2018|pp=4–5}} Decision-making is often handled in an anti-authoritarian way, with everyone having equal say in each decision, an approach known as [[horizontalism]].{{sfn|Williams|2019|pp=109–117}} Contemporary-era anarchists have been engaging with various [[grassroots]] movements that are more or less based on horizontalism, although not explicitly anarchist, respecting personal autonomy and participating in mass activism such as strikes and demonstrations. In contrast with the ''big-A anarchism'' of the classical era, the newly coined term ''small-a anarchism'' signals their tendency not to base their thoughts and actions on classical-era anarchism or to refer to [[classical anarchists]] such as [[Peter Kropotkin]] and [[Pierre-Joseph Proudhon]] to justify their opinions. Those anarchists would rather base their thought and praxis on their own experience which they will later theorize.{{sfn|Kinna|2019|pp=145–149}}

The decision-making process of small anarchist [[affinity group]]s plays a significant tactical role.{{sfn|Williams|2019|pp=109, 119}} Anarchists have employed various methods in order to build a rough consensus among members of their group without the need of a leader or a leading group. One way is for an individual from the group to play the role of facilitator to help achieve a consensus without taking part in the discussion themselves or promoting a specific point. Minorities usually accept rough consensus, except when they feel the proposal contradicts anarchist ethics, goals and values. Anarchists usually form small groups (5–20 individuals) to enhance autonomy and friendships among their members. These kinds of groups more often than not interconnect with each other, forming larger networks. Anarchists still support and participate in strikes, especially [[Wildcat strike action|wildcat strikes]] as these are leaderless strikes not organised centrally by a syndicate.{{sfn|Williams|2019|p=119–121}}

As in the past, newspapers and journals are used, and anarchists have gone [[online]] in the [[World Wide Web]] to spread their message. Anarchists have found it easier to create websites because of distributional and other difficulties, hosting electronic libraries and other portals.{{sfn|Williams|2019|pp=118–119}} Anarchists were also involved in developing various software that are available for free. The way these hacktivists work to develop and distribute resembles the anarchist ideals, especially when it comes to preserving users' privacy from state surveillance.{{sfn|Williams|2019|pp=120–121}}

Anarchists organize themselves to [[Squatting|squat]] and reclaim public spaces. During important events such as protests and when spaces are being occupied, they are often called [[Temporary Autonomous Zone]]s (TAZ), spaces where art, poetry, and [[surrealism]] are blended to display the anarchist ideal.{{sfnm|1a1=Kinna|1y=2019|1p=139|2a1=Mattern|2y=2019|2p=596|3a1=Williams|3y=2018|3pp=5–6}} As seen by anarchists, squatting is a way to regain urban space from the capitalist market, serving pragmatical needs and also being an exemplary direct action.{{sfnm|1a1=Kinna|1y=2012|1p=250|2a1=Williams|2y=2019|2p=119}} Acquiring space enables anarchists to experiment with their ideas and build social bonds.{{sfn|Williams|2019|p=122}} Adding up these tactics while having in mind that not all anarchists share the same attitudes towards them, along with various forms of protesting at highly symbolic events, make up a [[carnivalesque]] atmosphere that is part of contemporary anarchist vividity.{{sfn|Morland|2004|p=37–38}}

== Key issues ==
{{main|Issues in anarchism}}
<!-- In the interest of restricting article length, please limit this section to two or three short paragraphs and add any substantial information to the main Issues in anarchism article. Thank you. -->
As anarchism is a [[philosophy]] that embodies many diverse attitudes, tendencies, and schools of thought; disagreement over questions of values, ideology, and tactics is common. Its diversity has led to widely different uses of identical terms among different anarchist traditions which has created a number of [[definitional concerns in anarchist theory]]. The compatibility of [[Anarchism and capitalism|capitalism]],{{sfnm|1a1=Marshall|1y=1993|1p=565|2a1=Honderich|2y=1995|2p=31|3a1=Meltzer|3y=2000|3p=50|4a1=Goodway|4y=2006|4p=4|5a1=Newman|5y=2010|5p=53}} [[Anarchism and nationalism|nationalism]], and [[Anarchism and religion|religion]] with anarchism is widely disputed, and anarchism enjoys complex relationships with ideologies such as [[Anarchism and communism|communism]], [[collectivism]], [[Anarchism and Marxism|Marxism]], and [[trade unionism]]. Anarchists may be motivated by [[humanism]], [[God|divine authority]], [[enlightened self-interest]], [[Veganarchism|veganism]], or any number of alternative ethical doctrines. Phenomena such as [[civilisation]], [[technology]] (e.g. within [[anarcho-primitivism]]), and the [[Anarchism and statist democracy|democratic process]] may be sharply criticised within some anarchist tendencies and simultaneously lauded in others.{{sfn|De George|2005|pp=31–32}}

=== Gender, sexuality, and free love ===
{{Main|Anarchism and issues related to love and sex}}
[[File:Emilearmand01.jpg|left|thumb|upright|[[Émile Armand]] was a French individualist anarchist who propounded the virtues of free love in the Parisian anarchist milieu of the early 20th century]]
As gender and sexuality carry along them dynamics of hierarchy, many anarchists address, analyse, and oppose the suppression of one's autonomy imposed by gender roles.{{sfn|Nicholas|2019|p=603}}

[[File:AnarchaFeminism.png|thumb|upright=1.2|Collection of [[anarcha-feminist]] protests, symbols, and flags]]
Sexuality was not often discussed by classical anarchists but the few that did felt that an anarchist society would lead to sexuality naturally developing.{{sfn|Lucy|2020|p=162}} Sexual violence was a concern for anarchists such as [[Benjamin Tucker]], who opposed age of consent laws, believing they would benefit predatory men.{{sfn|Lucy|2020|p=178}} A historical current that arose and flourished during 1890 and 1920 within anarchism was [[free love]]. In contemporary anarchism, this current survives as a tendency to support [[polyamory]] and [[queer anarchism]].{{sfnm|1a1=Nicholas|1y=2019|1p=611|2a1=Jeppesen|2a2=Nazar|2y=2012|2pp=175–176}} Free love advocates were against marriage, which they saw as a way of men imposing authority over women, largely because marriage law greatly favoured the power of men. The notion of free love was much broader and included a critique of the established order that limited women's sexual freedom and pleasure.{{sfn|Jeppesen|Nazar|2012|pp=175–176}} Those free love movements contributed to the establishment of communal houses, where large groups of travelers, anarchists and other activists slept in beds together.{{sfn|Jeppesen|Nazar|2012|p=177}} Free love had roots both in Europe and the United States; however, some anarchists struggled with the jealousy that arose from free love.{{sfn|Jeppesen|Nazar|2012|pp=175–177}} Anarchist feminists were advocates of free love, against marriage, and pro-choice (utilising a contemporary term), and had a similar agenda. Anarchist and non-anarchist feminists differed on [[suffrage]] but were supportive of one another.{{sfn|Kinna2019|pp=166–167}}

During the second half of the 20th century, anarchism intermingled with the [[second wave of feminism]], radicalising some currents of the feminist movement and being influenced as well. By the latest decades of the 20th century, anarchists and feminists were advocating for the rights and autonomy of women, gays, queers and other marginalised groups, with some feminist thinkers suggesting a fusion of the two currents.{{sfn|Nicholas|2019|pp=609–611}} With the [[third wave of feminism]], sexual identity and compulsory heterosexuality became a subject of study for anarchists, yielding a [[post-structuralist]] critique of sexual normality.{{sfn|Nicholas|2019|pp=610–611}} Some anarchists distanced themselves from this line of thinking, suggesting that it leaned towards an individualism that was dropping the cause of social liberation.{{sfn|Nicholas|2019|pp=616–617}}

=== Anarchism and education ===
{{main|Anarchism and education}}
{|class="wikitable" style="border: none; background: none; float: right;"
|+ Anarchist vs. statist perspectives on education<br><small>Ruth Kinna (2019){{sfn|Kinna|2019|p=97}}</small>
|-
!scope="col"|
!scope="col"|Anarchist education
!scope="col"|State education
|-
|Concept || Education as self-mastery || Education as service
|-
|Management || Community based || State run
|-
|Methods || Practice-based learning || Vocational training
|-
|Aims || Being a critical member of society || Being a productive member of society
|}
The interest of anarchists in education stretches back to the first emergence of classical anarchism. Anarchists consider proper education, one which sets the foundations of the future autonomy of the individual and the society, to be an act of [[Mutual aid (organization theory)|mutual aid]].{{sfnm|1a1=Kinna|1y=2019|1pp=83–85|2a2=Suissa|2y=2019|2pp=514–515, 520}} Anarchist writers such as [[William Godwin]] (''[[Political Justice]]'') and [[Max Stirner]] ("[[The False Principle of Our Education]]") attacked both state education and private education as another means by which the ruling class replicate their privileges.{{sfnm|1a1=Suissa|1y=2019|1pp=514, 521|2a1=Kinna|2y=2019|2pp=83–86|3a1=Marshall|3y=1993|3p=222}}

In 1901, Catalan anarchist and free thinker [[Francisco Ferrer]] established the [[Escuela Moderna]] in Barcelona as an opposition to the established education system which was dictated largely by the Catholic Church.{{sfn|Suissa|2019|pp=511–512}} Ferrer's approach was secular, rejecting both state and church involvement in the educational process whilst giving pupils large amounts of autonomy in planning their work and attendance. Ferrer aimed to educate the working class and explicitly sought to foster [[class consciousness]] among students. The school closed after constant harassment by the state and Ferrer was later arrested. Nonetheless, his ideas formed the inspiration for a series of [[Modern School (United States)|modern schools]] around the world.{{sfn|Suissa|2019|pp=511–514}} Christian anarchist [[Leo Tolstoy]], who published the essay ''Education and Culture'', also established a similar school with its founding principle being that "for education to be effective it had to be free."{{sfn|Suissa|2019|pp=517–518}} In a similar token, A. S. Neill founded what became the [[Summerhill School]] in 1921, also declaring being free from coercion.{{sfn|Suissa|2019|pp=518–519}}

Anarchist education is based largely on the idea that a child's right to develop freely and without manipulation ought to be respected and that rationality would lead children to morally good conclusions; however, there has been little consensus among anarchist figures as to what constitutes manipulation. Ferrer believed that moral indoctrination was necessary and explicitly taught pupils that equality, liberty and social justice were not possible under capitalism, along with other critiques of government and nationalism.{{sfnm|1a1=Avrich|1y=1980|1pp=3–33|2a1=Suissa|2y=2019|2pp=519–522}}

Late 20th century and contemporary anarchist writers ([[Paul Goodman]], [[Herbert Read]], and [[Colin Ward]]) intensified and expanded the anarchist critique of state education, largely focusing on the need for a system that focuses on children's creativity rather than on their ability to attain a career or participate in [[consumerism]] as part of a consumer society.{{sfn|Kinna|2019|pp=89–96}} Contemporary anarchists such as Ward claim that state education serves to perpetuate socioeconomic inequality.{{sfn|Ward|1973|pp=39–48}}

While few anarchist education institutions have survived to the modern-day, major tenets of anarchist schools, among them respect for child autonomy and relying on reasoning rather than indoctrination as a teaching method, have spread among mainstream educational institutions. Judith Suissa names three schools as explicitly anarchists schools, namely the Free Skool Santa Cruz in the United States which is part of a wider American-Canadian network of schools, the Self-Managed Learning College in Brighton, England, and the Paideia School in Spain.{{sfn|Suissa|2019|pp=523–526}}

=== Anarchism and the state ===
<!-- Important! Strive to explain how anarchists perceive authority and oppression and why they reject them. Jun (2019), p. 41. -->
Objection to the [[State (polity)|state]] and its institutions is a ''[[sine qua non]]'' of anarchism.{{sfnm|1a1=Carter|1y=1971|1p=14|2a1=Jun|2y=2019|2pp=29–30}} Anarchists consider the state as a tool of domination and believe it to be illegitimate regardless of its political tendencies. Instead of people being able to control the aspects of their life, major decisions are taken by a small elite. Authority ultimately rests solely on power, regardless of whether that power is [[Open government|open]] or [[Transparency (behavior)|transparent]], as it still has the ability to coerce people. Another anarchist argument against states is that the people constituting a government, even the most altruistic among officials, will unavoidably seek to gain more power, leading to corruption. Anarchists consider the idea that the state is the collective will of the people to be an unachievable fiction due to the fact that the [[ruling class]] is distinct from the rest of society.{{sfn|Jun|2019|pp=32–38}}

Specific anarchist attitudes towards the state vary. [[Robert Paul Wolff]] believed that the tension between authority and autonomy would mean the state could never be legitimate. Bakunin saw the state as meaning "coercion, domination by means of coercion, camouflaged if possible but unceremonious and overt if need be." [[A. John Simmons]] and [[Leslie Green (philosopher)|Leslie Green]], who leaned toward philosophical anarchism, believed that the state could be legitimate if it is governed by consensus, although they saw this as highly unlikely.{{sfnm|1a1=Wendt|1y=2020|1p=2|2a1=Ashwood|2y=2018|2p=727}} Beliefs on how to abolish the state also differ.{{Sfn|Ashwood|2018|p=735}}

=== Anarchism and the arts ===
{{main|Anarchism and the arts}}
[[File:Apple Harvest by Camille Pissarro.jpg|thumb|340px|''Les chataigniers a Osny'' (1888) by anarchist painter Camille Pissarro is a notable example of blending anarchism and the arts.{{sfn|Antliff|1998|p=99}}]]
The connection between anarchism and art was quite profound during the classical era of anarchism, especially among artistic currents that were developing during that era such as futurists, surrealists and others.{{sfn|Mattern|2019|p=592}} In literature, anarchism was mostly associated with the [[New Apocalyptics]] and the [[neo-romanticism]] movement.{{snf|Gifford|2019|p=577}} In music, anarchism has been associated with music scenes such as punk.{{sfnm|1a1=Marshall|1y=1993|1pp=493–494|2a1=Dunn|2y=2012|3a1=Evren|3a2=Kinna|3a3=Rouselle|3y=2013|p=138}} Anarchists such as [[Leo Tolstoy]] and [[Herbert Read]] stated that the border between the artist and the non-artist, what separates art from a daily act, is a construct produced by the alienation caused by capitalism and it prevents humans from living a joyful life.{{sfn|Mattern|2019|pp=592–593}}

Other anarchists advocated for or used art as a means to achieve anarchist ends.{{sfn|Mattern|2019|p=593}} In his book ''Breaking the Spell: A History of Anarchist Filmmakers, Videotape Guerrillas, and Digital Ninjas'', Chris Robé claims that "anarchist-inflected practices have increasingly structured movement-based video activism."{{sfn|Robé|2017|p=44}} Throughout the 20th century, many prominent anarchists ([[Peter Kropotkin]], [[Emma Goldman]], [[Gustav Landauer]] and [[Camillo Berneri]]) and publications such as ''[[Anarchy (magazine)|Anarchy]]'' wrote about matters pertaining to the arts.{{sfn|Miller|Dirlik|Rosemont|Augustyn|2019|p=1}}

Three overlapping properties made art useful to anarchists. It could depict a critique of existing society and hierarchies, serve as a prefigurative tool to reflect the anarchist ideal society and even turn into a means of direct action such as in protests. As it appeals to both emotion and reason, art could appeal to the whole human and have a powerful effect.{{sfn|Mattern|2019|pp=593–596}} The 19th-century [[neo-impressionist]] movement had an ecological aesthetic and offered an example of an anarchist perception of the road towards socialism.{{sfn|Antliff|1998|p=78}} In ''Les chataigniers a Osny'' by anarchist painter [[Camille Pissarro]], the blending of aesthetic and social harmony is prefiguring an ideal anarchistic agrarian community.{{sfn|Antliff|1998|p=99}}

== Analysis ==
The most common critique of anarchism is that humans cannot self-govern and so a state is necessary for human survival. Philosopher [[Bertrand Russell]] supported this critique, stating that "[p]eace and war, tariffs, regulations of sanitary conditions and the sale of noxious drugs, the preservation of a just system of distribution: these, among others, are functions which could hardly be performed in a community in which there was no central government."{{sfn|Krimerman|Perry|1966|p=494}} Another common criticism of anarchism is that it fits a world of isolation in which only the small enough entities can be self-governing; a response would be that major anarchist thinkers advocated [[anarchist federalism]].{{sfn|Ward|2004|p=78}}

Philosophy lecturer Andrew G. Fiala composed a list of common arguments against anarchism which includes critiques such as that anarchism is innately related to violence and destruction, not only in the pragmatic world, such as at protests, but in the world of ethics as well. Secondly, anarchism is evaluated as unfeasible or utopian since the state cannot be defeated practically. This line of arguments most often calls for political action within the system to reform it. The third argument is that anarchism is self-contradictory. While it advocates for no-one to ''archiei'', if accepted by the many, then anarchism would turn into the ruling political theory. In this line of criticism also comes the self-contradiction that anarchism calls for collective action whilst endorsing the autonomy of the individual, hence no collective action can be taken. Lastly, Fiala mentions a critique towards philosophical anarchism of being ineffective (all talk and thoughts) and in the meantime capitalism and bourgeois class remains strong.{{sfn|Fiala|2017|loc="4. Objections and Replies"}}

Philosophical anarchism has met the criticism of members of academia following the release of pro-anarchist books such as [[A. John Simmons]]' ''Moral Principles and Political Obligations''.{{sfn|Klosko|1999|p=536}} Law professor William A. Edmundson authored an essay to argue against three major philosophical anarchist principles which he finds fallacious. Edmundson says that while the individual does not owe the state a duty of obedience, this does not imply that anarchism is the inevitable conclusion and the state is still morally legitimate.{{sfnm|1a1=Klosko|1y=1999|1p=536|2a1=Kristjánsson|2y=2000|2p=896}} In ''The Problem of Political Authority'', [[Michael Huemer]] defends philosophical anarchism,{{sfn|Dagger|2018|p=35}} claiming that "political authority is a moral illusion."{{sfn|Rogers|2020|p=}}

One of the earliest criticisms is that anarchism defies and fails to understand the biological inclination to authority.{{sfn|Ferguson|1886}} [[Joseph Raz]] states that the acceptance of authority implies the belief that following their instructions will afford more success.{{sfn|Gans|1992|p=37}} Raz believes that this argument is true in following both authorities' successful and mistaken instruction.{{sfn|Gans|1992|p=38}} Anarchists reject this criticism because challenging or disobeying authority does not entail the disappearance of its advantages by acknowledging authority such as doctors or lawyers as reliable, nor does it involve a complete surrender of independent judgment.{{sfn|Gans|1992|pp=34, 38}} Anarchist perception of human nature, rejection of the state, and commitment to social revolution has been criticised by academics as naive, overly simplistic, and unrealistic, respectively.{{sfn|Brinn|2020|p=206}} Classical anarchism has been criticised for relying too heavily on the belief that the abolition of the state will lead to human cooperation prospering.{{sfn|Lucy|2020|p=162}}

[[Friedrich Engels]], considered to be one of the principal founders of [[Marxism]], criticised anarchism's anti-authoritarianism as inherently counter-revolutionary because in his view a revolution is by itself authoritarian.{{sfn|Tucker|1978}} Academic [[John Molyneux (academic)|John Molyneux]] writes in his book ''Anarchism: A Marxist Criticism'' that "anarchism cannot win", believing that it lacks the ability to properly implement its ideas.{{sfn|Dodds|2011}} The Marxist criticism of anarchism is that it has a utopian character because all individuals should have anarchist views and values. According to the Marxist view, that a social idea would follow directly from this human ideal and out of the free will of every individual formed its essence. Marxists state that this contradiction was responsible for their inability to act. In the anarchist vision, the conflict between liberty and equality was resolved through coexistence and intertwining.{{sfn|Baár|Falina|Janowski|Kopeček|2016|p=488}}

== See also ==
{{Portal|Anarchism|Libertarianism}}
* [[:Category:Anarchism by country|Anarchism by country]]
* [[Governance without government]]
* [[List of anarchist political ideologies]]
* [[List of books about anarchism]]

== References ==
=== Citations ===
{{reflist|25em}}

=== Notes ===
{{reflist|group=nb}}

=== Sources ===
==== Primary sources ====
{{refbegin|35em|indent=yes}}
* {{cite book|last=Bakunin|first=Mikhail|author-link=Mikhail Bakunin|title=Statism and Anarchy|title-link=Statism and Anarchy|year=1990|orig-year=1873|publisher=Cambridge University Press|location=Cambridge, England |series=Cambridge Texts in the History of Political Thought|translator-last=Shatz|translator-first=Marshall|isbn=978-0-521-36182-8|oclc=20826465|lccn=89077393|doi=10.1017/CBO9781139168083|editor1-last=Shatz|editor1-first=Marshall}}
{{refend}}

==== Secondary sources ====
{{refbegin|35em|indent=yes}}
* {{cite journal|last1=Angelbeck|first1=Bill|last2=Grier|first2=Colin|year=2012|title=Anarchism and the Archaeology of Anarchic Societies: Resistance to Centralization in the Coast Salish Region of the Pacific Northwest Coast|journal=Current Anthropology|volume=53|issue=5|pages=547–587|doi=10.1086/667621|url=https://arcabc.ca/islandora/object/dc%3A312/datastream/PDF/view}}
* {{cite journal|last=Antliff|first=Mark|year=1998|title=Cubism, Futurism, Anarchism: The 'Aestheticism' of the "Action d'art" Group, 1906–1920|journal=Oxford Art Journal|volume=21|issue=2|pages=101–120|jstor=1360616|doi=10.1093/oxartj/21.2.99}}
* {{cite journal|last=Anderson|first=Benedict|author-link=Benedict Anderson|title=In the World-Shadow of Bismarck and Nobel|journal=[[New Left Review]]|volume=2|issue=28|pages=85–129|year=2004|url=http://newleftreview.org/II/28/benedict-anderson-in-the-world-shadow-of-bismarck-and-nobel|access-date=7 January 2016|archive-url=https://web.archive.org/web/20151219130121/http://newleftreview.org/II/28/benedict-anderson-in-the-world-shadow-of-bismarck-and-nobel|archive-date=19 December 2015}}
* {{cite book|last=Arvidsson|first=Stefan|author-link=Stefan Arvidsson|year=2017|title=The Style and Mythology of Socialism: Socialist Idealism, 1871–1914|edition=1st|location=London|publisher=Routledge|isbn=9780367348809}}
* {{cite journal|last=Ashwood|first=Loka|year=2018|title=Rural Conservatism or Anarchism? The Pro‐state, Stateless, and Anti‐state Positions|journal=Rural Sociology|volume=83|issue=4|pages=717–748|doi=10.1111/ruso.12226}}
* {{cite book|last=Avrich|first=Paul|author-link=Paul Avrich|title=[[Anarchist Voices: An Oral History of Anarchism in America]]|year=1996|publisher=Princeton University Press|isbn=978-0-691-04494-1}}
* {{cite book|last=Avrich|first=Paul|author-link=Paul Avrich|author-mask=2|title=The Russian Anarchists|publisher=[[AK Press]]|location=Stirling|year=2006|isbn=978-1-904859-48-2|title-link=The Russian Anarchists}}
* {{cite book|last=Avrich|first=Paul|author-link=Paul Avrich|author-mask=2|title=[[The Modern School Movement: Anarchism and Education in the United States]]|publisher=Princeton University Press|year=1980|isbn=978-1-4008-5318-2|pages=3–33|oclc=489692159}}
* {{cite book|last1=Baár|first1=Monika|last2=Falina|first2=Maria|last3=Janowski|first3=Maciej|last4=Kopeček|first4=Michal|last5=Trencsényi|first5=Balázs Trencsényi|year=2016|title=A History of Modern Political Thought in East Central Europe: Negotiating Modernity in the 'Long Nineteenth Century'|volume=I|location=Oxford|publisher=Oxford University Press|isbn=978-0-19-105695-6}}
* {{cite book|last=Bantman|first=Constance|chapter=The Era of Propaganda by the Deed|pages=371–388|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Bates|first=David|editor=Paul Wetherly|title=Political Ideologies|chapter-url=https://books.google.com/books?id=uXfJDgAAQBAJ&pg=PA128|year=2017|publisher=Oxford University Press|isbn=978-0-19-872785-9|chapter=Anarchism}}
* {{cite book|last=Bolloten|first=Burnett|author-link=Burnett Bolloten|title=The Spanish Civil War: Revolution and Counterrevolution|publisher=University of North Carolina Press|year=1984|isbn=978-0-8078-1906-7}}
* {{cite journal|last=Brinn|first=Gearóid|year=2020|title=Smashing the State Gently: Radical Realism and Realist Anarchism|journal=European Journal of Political Theory|volume=19|issue=2|pages=206–227|doi=10.1177/1474885119865975|s2cid=202278143}}
* {{cite book|last=Brooks|first=Frank H.|year=1994|title=The Individualist Anarchists: An Anthology of Liberty (1881–1908)|publisher=Transaction Publishers|isbn=978-1-56000-132-4}}
* {{cite book|last=Carlson|first=Andrew R.|year=1972|url=https://libcom.org/files/andrew-r-carlson-anarchism-in-germany-volume-i-the-early-movement-1.pdf|title=Anarchism in Germany; Vol. 1: The Early Movement|location=Metuchen, New Jersey|publisher=Scarecrow Press|isbn=978-0-81080-484-5}}
* {{cite book|last=Carter|first=April|author-link=April Carter|title=The Political Theory of Anarchism|url=https://books.google.com/books?id=3mlWPgAACAAJ|year=1971|publisher=Routledge|isbn=978-0-415-55593-7}}
* {{cite journal|last=Carter|first=April|title=Anarchism and violence|journal=Nomos|volume=19|pages=320–340|year=1978|publisher=American Society for Political and Legal Philosophy|jstor=24219053}}
* {{cite book|editor1-last=Chaliand|editor1-first=Gerard|editor2-last=Blin|editor2-first=Arnaud|title=The History of Terrorism: From Antiquity to Al-Quaeda|publisher=University of California Press|location=Berkeley; Los Angeles; London|year=2007|isbn=978-0-520-24709-3|oclc=634891265|url-access=registration|url=https://archive.org/details/historyofterrori00grar}}
* {{cite book|last=Chomsky|first=Noam|author-link=Noam Chomsky|editor-last=Pateman|editor-first=Barry|title=Chomsky on Anarchism|year=2005|isbn=978-1-904859-26-0|publisher=AK Press|location=Oakland}}
* {{cite encyclopedia|last=Cohn|first=Jesse|editor-last=Ness|editor-first=Immanuel|title=Anarchism|encyclopedia=The International Encyclopedia of Revolution and Protest|pages=1–11|year=2009|isbn=978-1-4051-9807-3|publisher=John Wiley & Sons|location=Oxford|doi=10.1002/9781405198073.wbierp0039}}
* {{cite book|last=Dagger|first=Tristan J.|year=2018|title=Playing Fair: Political Obligation and the Problems of Punishment|location=Oxford|publisher=Oxford University Press|isbn=978-0-19-938883-7}}
* {{cite book|last=Dirlik|first=Arif|title=Anarchism in the Chinese Revolution|publisher=University of California Press|location=Berkeley, CA|year=1991|isbn=978-0-520-07297-8}}
* {{cite magazine|last=Dodds|first=Jonathan|date=October 2011|url=http://socialistreview.org.uk/362/anarchism-marxist-criticism|title=Anarchism: A Marxist Criticism|magazine=Socialist Review|access-date=31 July 2020}}
* {{cite book|last=Dodson|first=Edward|title=The Discovery of First Principles|volume=2|publisher=Authorhouse|year=2002|isbn=978-0-595-24912-1}}
* {{cite journal|last=Dunn|first=Kevin|date=August 2012|title=Anarcho-Punk and Resistance in Everyday Life|journal=Punk & Post-Punk|publisher=Intellect|volume=1|issue=2|pages=201–218|doi=10.1386/punk.1.2.201_1}}
* {{cite book|last=Egoumenides|first=Magda|year=2014|url=https://books.google.com/books?id=4qEMBAAAQBAJ|title=Philosophical Anarchism and Political Obligation|location=New York|publisher=Bloomsbury Publishing USA|isbn=978-1-4411-2445-6}}
* {{cite book|last=Evren|first=Süreyyya|author-link=Süreyyya Evren|chapter=How New Anarchism Changed the World (of Opposition) after Seattle and Gave Birth to Post-Anarchism|pages=1–19|editor-last1=Rousselle|editor-first1=Duane|editor-last2=Evren|editor-first2=Süreyyya|title=Post-Anarchism: A Reader|year=2011|isbn=978-0-7453-3086-0|publisher=[[Pluto Press]]}}
* {{cite book|last1=Evren|first1=Süreyyya|last2=Kinna|first2=Ruth|last3=Rouselle|first3=Duane|year=2013|title=Blasting the Canon|location=Santa Barba, California|publisher=[[Punctum Books]]|isbn=9780615838625}}
* {{cite journal|last=Ferguson|first=Francis L.|date=August 1886|title=The Mistakes of Anarchism|journal=The North American Review|publisher=University of Northern Iowa|volume=143|issue=357|pages=204–206|issn=0029-2397|jstor=25101094}}
* {{cite book|last=Fernández|first=Frank|title=Cuban Anarchism: The History of A Movement|year=2009|orig-year=2001|publisher=Sharp Press}}
* {{cite journal|last1=Franks|first1=Benjamin|author-link=Benjamin Franks (philosopher)|editor-last1=Freeden|editor-first1=Michael|editor-last2=Stears|editor-first2=Marc|title=Anarchism|journal=The Oxford Handbook of Political Ideologies|pages=385–404|date=August 2013|publisher=Oxford University Press|doi=10.1093/oxfordhb/9780199585977.013.0001}}
* {{cite book|last1=Franks|first1=Benjamin|author-link=Benjamin Franks (philosopher)|author-mask=2|chapter=Anarchism and Ethics|pages=549–570|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last=Gabardi|first=Wayne|year=1986|volume=80|issue=1|pages=300–302|doi=10.2307/1957102|jstor=446800|title=Anarchism. By David Miller. (London: J. M. Dent and Sons, 1984. Pp. 216. £10.95.)|journal=American Political Science Review}}
* {{cite book|last=Gans|first=Chaim|year=1992|title=Philosophical Anarchism and Political Disobedience|edition=reprint|location=Cambridge|publisher=Cambridge University Press|isbn=978-0-521-41450-0}}
* {{cite book|last1=Gay|first1=Kathlyn|last2=Gay|first2=Martin|title=Encyclopedia of Political Anarchy|year=1999|isbn=978-0-87436-982-3|publisher=ABC-CLIO}}
* {{cite book|last=Gifford|first=James|chapter=Literature and Anarchism|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Goodway|first=David|author-link=David Goodway|title=[[Anarchist Seeds Beneath the Snow]]|publisher=Liverpool Press|year=2006|isbn=978-1-84631-025-6}}
* {{cite book|last=Graham|first=Robert|title=Anarchism: a Documentary History of Libertarian Ideas: from Anarchy to Anarchism|publisher=Black Rose Books|location=Montréal|year=2005|isbn=978-1-55164-250-5|author-link=Robert Graham (historian)|url=http://robertgraham.wordpress.com/anarchism-a-documentary-history-of-libertarian-ideas-volume-one-from-anarchy-to-anarchism-300ce-1939/|access-date=5 March 2011|archive-url=https://web.archive.org/web/20101130131904/http://robertgraham.wordpress.com/anarchism-a-documentary-history-of-libertarian-ideas-volume-one-from-anarchy-to-anarchism-300ce-1939/|archive-date=30 November 2010}}
* {{cite book|last=Graham|first=Robert|author-link=Robert Graham (historian)|author-mask=2|chapter=Anarchism and the First International|pages=325–342|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|year=2019|publisher=Springer|isbn=978-3-319-75620-2}}
* {{cite book|last=Guérin|first=Daniel|author-link=Daniel Guérin|year=1970|url=http://theanarchistlibrary.org/library/daniel-guerin-anarchism-from-theory-to-practice|title=Anarchism: From Theory to Practice|publisher=Monthly Review Press|isbn=978-0-85345-128-0}}
* {{cite book|last1=Harrison|first1=Kevin|last2=Boyd|first2=Tony|title=Understanding Political Ideas and Movements|url=https://books.google.com/books?id=5qrJCgAAQBAJ|date=5 December 2003|publisher=Manchester University Press|isbn=978-0-7190-6151-6}}
* {{cite journal|last=Harmon|first=Christopher C.|year=2011|title=How Terrorist Groups End: Studies of the Twentieth Century|journal=Connections|volume=10|issue=2|pages=51–104|jstor=26310649}}
* {{cite book|last=Heywood|first=Andrew|author-link=Andrew Heywood|year=2017|title=Political Ideologies: An Introduction|url=https://books.google.com/books?id=Sy8hDgAAQBAJ&pg=PA146|edition=6th|publisher=Macmillan International Higher Education|isbn=978-1-137-60604-4}}
* {{cite book|last=Honderich|first=Ted|title=The Oxford Companion to Philosophy|url=https://archive.org/details/oxfordcompaniont00hond|url-access=registration|year=1995|publisher=Oxford University Press|isbn=978-0-19-866132-0}}
* {{cite news|last=Imrie|first=Doug|title=The Illegalists|year=1994|work=Anarchy: A Journal of Desire Armed|url=http://recollectionbooks.com/siml/library/illegalistsDougImrie.htm|url-status=dead|archive-url=https://web.archive.org/web/20150908072801/http://recollectionbooks.com/siml/library/illegalistsDougImrie.htm|archive-date=8 September 2015|access-date=9 December 2010}}
* {{cite book|last1=Jennings|first1=Jeremy|editor-last1=Eatwell|editor-first1=Roger|editor-last2=Wright|editor-first2=Anthony|chapter=Anarchism|title=Contemporary Political Ideologies|pages=127–146|year=1993|isbn=978-0-86187-096-7|publisher=Pinter|location=London}}
* {{cite book|last=Jennings|first=Jeremy|year=1999|chapter=Anarchism|editor-last1=Eatwell|editor-first1=Roger|editor-last2=Wright|editor-first2=Anthony|title=Contemporary Political Ideologies|edition=reprinted, 2nd|location=London|publisher=A & C Black|isbn=978-0-8264-5173-6}}
* {{cite book|last1=Jeppesen|first1=Sandra|last2=Nazar|first2=Holly|chapter=Genders and Sexualities in Anarchist Movements|editor=[[Ruth Kinna]]|title=The Bloomsbury Companion to Anarchism|url=https://books.google.com/books?id=dNuoAwAAQBAJ|date=28 June 2012|publisher=Bloomsbury Publishing|isbn=978-1-4411-4270-2}}
* {{cite book|last1=Johnson|first1=Charles|editor-last1=Long|editor-first1=Roderick T.|editor-last2=Machan|editor-first2=Tibor R.|editor-link2=Tibor Machan|chapter=Liberty, Equality, Solidarity Toward a Dialectical Anarchism|title=Anarchism/Minarchism: Is a Government Part of a Free Country?|pages=155–188|year=2008|isbn=978-0-7546-6066-8|publisher=Ashgate}}
* {{cite book|title=[[The Anarchists (book)|The Anarchists]]|last=Joll|first=James|author-link=James Joll|year=1964|publisher=Harvard University Press|isbn=978-0-674-03642-0}}
* {{cite journal|last1=Jun|first1=Nathan|title=Anarchist Philosophy and Working Class Struggle: A Brief History and Commentary|journal=[[WorkingUSA]]|volume=12|issue=3|pages=505–519|date=September 2009|language=en|doi=10.1111/j.1743-4580.2009.00251.x|issn=1089-7011}}<!-- to revisit, p. 508+ -->
* {{cite book|last=Jun|first=Nathan|chapter=The State|pages=27–47|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last=Kahn|first=Joseph|title=Anarchism, the Creed That Won't Stay Dead; The Spread of World Capitalism Resurrects a Long-Dormant Movement|year=2000|journal=[[The New York Times]]|issue=5 August}}
* {{cite book|last=Kinna|first=Ruth|author-link=Ruth Kinna|title=The Bloomsbury Companion to Anarchism|year=2012|publisher=Bloomsbury Academic|isbn=978-1-62892-430-5}}
* {{cite book|last=Kinna|first=Ruth|author-link=Ruth Kinna|title=The Government of No One, The Theory and Practice of Anarchism|publisher=[[Penguin Random House]]|url=https://books.google.com/books?id=xzeGDwAAQBAJ|year=2019|isbn=978-0-241-39655-1}}
* {{cite journal|last=Klosko|first=George|title=More than Obligation – William A. Edmundson: Three Anarchical Fallacies: An Essay on Political Authority|journal=The Review of Politics|volume=61|issue=3|year=1999|issn=1748-6858|doi=10.1017/S0034670500028989|pages=536–538}}
* {{cite book|last=Klosko|first=George|title=Political Obligations|url=https://books.google.com/books?id=ToHmfIj8d_gC|year=2005|publisher=Oxford University Press|isbn=978-0-19-955104-0}}
* {{Cite book |editor-last1=Krimerman |editor-first1=Leonard I. |editor-last2=Perry |editor-first2=Lewis |title=Patterns of Anarchy: A Collection of Writings on the Anarchist Tradition |date=1966 |url=http://archive.org/details/patternsofanarch00krim |language=en |publisher=Anchor Books |location=Garden City, NY }}
* {{cite journal|last=Kristjánsson|first=Kristján|title=Three Anarchical Fallacies: An Essay on Political Authority by William A. Edmundson|journal=Mind|volume=109|issue=436|pages=896–900|year=2000|jstor=2660038}}
* {{cite book|last=Laursen|first=Ole Birk|chapter=Anti-Imperialism|pages=149–168|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|year=2019|publisher=Springer|isbn=978-3-319-75620-2}}
* {{cite journal|last=Levy|first=Carl|s2cid=144317650|author-link=Carl Levy (political scientist)|title=Social Histories of Anarchism|journal=Journal for the Study of Radicalism|volume=4|issue=2|date=8 May 2011|issn=1930-1197|doi=10.1353/jsr.2010.0003|pages=1–44}}
* {{Cite book |editor-last1=Levy |editor-first1=Carl |editor-link1=Carl Levy (political scientist) |editor-last2=Adams |editor-first2=Matthew S. |title=The Palgrave Handbook of Anarchism |date=2018 |isbn=978-3-319-75619-6 |publisher=Palgrave Macmillan }}
* {{cite book|last=Long|first=Roderick T.|editor-last1=Gaud|editor-first1=Gerald F.|editor-last2=D'Agostino|editor-first2=Fred|title=The Routledge Companion to Social and Political Philosophy|url=https://books.google.com/books?id=z7MzEHaJgKAC|year=2013|publisher=Routledge|isbn=978-0-415-87456-4}}
* {{cite book|last=Lucy|first=Nicholas|year=2020|chapter-url=https://books.google.com/books?id=-t7KDwAAQBAJ|chapter=Anarchism and Sexuality|title=The SAGE Handbook of Global Sexualities|publisher=SAGE Publishing|pages=160–183|isbn=978-1-5297-2194-2}}
* {{cite journal|last1=Lutz|first1=James M.|last2=Ulmschneider|first2=Georgia Wralstad|year=2019|title=Civil Liberties, National Security and U.S. Courts in Times of Terrorism|journal=Perspectives on Terrorism|volume=13|issue=6|pages=43–57|jstor=26853740}}
* {{cite book|last=Marshall|first=Peter|author-link=Peter Marshall (author)|title=[[Demanding the Impossible: A History of Anarchism]]|year=1992|isbn=978-0-00-217855-6|publisher=Harper Collins|location=London}}
* {{cite book|last=Marshall|first=Peter|author-link=Peter Marshall (author)|author-mask=2|title=Demanding the Impossible: A History of Anarchism|year=1993|publisher=PM Press|place=Oakland, California|isbn=978-1-60486-064-1}}
* {{cite book|last=Mattern|first=Mark|chapter=Anarchism and Art|pages=589–602|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Mayne|first=Alan James|url=https://books.google.com/books?id=6MkTz6Rq7wUC&q=Communist+anarchism+believes+in+collective+ownership&pg=PA131|title=From Politics Past to Politics Future: An Integrated Analysis of Current and Emergent Paradigms|year=1999|publisher=Greenwood Publishing Group|access-date=20 September 2010|isbn=978-0-275-96151-0}}
* {{cite book|last=McLaughlin|first=Paul|title=Anarchism and Authority: A Philosophical Introduction to Classical Anarchism|publisher=[[Ashgate Publishing|Ashgate]]|location=Aldershot|date=28 November 2007|isbn=978-0-7546-6196-2}}
* {{cite book|last1=Morland|first1=Dave|chapter=Anti-capitalism and poststructuralist anarchism|editor1=Jonathan Purkis|editor2=James Bowen|pages=23–38|title=Changing Anarchism: Anarchist Theory and Practice in a Global Age|url=https://books.google.com/books?id=etb2UFzCBv4C|year=2004|publisher=Manchester University Press|isbn=978-0-7190-6694-8}}
* {{cite book|last=Meltzer|first=Albert|author-link=Albert Meltzer|title=Anarchism: Arguments For and Against|url=https://archive.org/details/anarchism00albe|url-access=registration|date=1 January 2000|publisher=AK Press|isbn=978-1-873176-57-3}}
* {{cite book|last=Morris|first=Brian|title=Bakunin: The Philosophy of Freedom|url=https://books.google.com/books?id=GJXy5eCpPawC|date=January 1993|publisher=Black Rose Books|isbn=978-1-895431-66-7}}
* {{cite book|last=Morris|first=Brian|author-mask=2|year=2015|title=Anthropology, Ecology, and Anarchism: A Brian Morris Reader|others=Marshall, Peter|edition=illustrated|location=Oakland|publisher=PM Press|isbn=978-1-60486-093-1}}
* {{cite book|last=Morris|first=Christopher W.|title=An Essay on the Modern State|url=https://books.google.com/books?id=uuyJ9Bw8w7QC|year=2002|publisher=Cambridge University Press|isbn=978-0-521-52407-0}}
* {{cite journal|last=Moynihan|first=Colin|title=Book Fair Unites Anarchists. In Spirit, Anyway|year=2007|journal=The New York Times|issue=16 April}}
* {{cite book|last=Moya|first=Jose C|editor=Geoffroy de Laforcade|others=Kirwin R. Shaffer|title=In Defiance of Boundaries: Anarchism in Latin American History|chapter-url=https://books.google.com/books?id=ikt6AQAACAAJ|year=2015|publisher=[[University Press of Florida]]|isbn=978-0-8130-5138-3|chapter=Transference, culture, and critique The Circulation of Anarchist Ideas and Practices}}
* {{cite journal|last=Nesser|first=Petter|year=2012|title=Research Note: Single Actor Terrorism: Scope, Characteristics and Explanations|journal=Perspectives on Terrorism|volume=6|issue=6|pages=61–73|jstor=26296894}}
* {{cite book|last=Nettlau|first=Max|author-link=Max Nettlau|title=[[A Short History of Anarchism]]|year=1996|publisher=Freedom Press|isbn=978-0-900384-89-9}}
* {{cite book|last=Newman|first=Michael|year=2005|title=Socialism: A Very Short Introduction|location=Oxford|publisher=Oxford University Press|isbn=978-0-19-280431-0}}
* {{cite book|last=Newman|first=Saul|title=The Politics of Postanarchism|url=https://books.google.com/books?id=SiqBiViUsOkC&pg=PA43|year=2010|publisher=Edinburgh University Press|isbn=978-0-7486-3495-8}}
* {{cite book|last=Nicholas|first=Lucy|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=Springer|isbn=978-3-319-75620-2|chapter=Gender and Sexuality}}
* {{cite journal|last=Norris|first=Jesse J.|year=2020|title=Idiosyncratic Terrorism: Disaggregating an Undertheorized Concept|journal=Perspectives on Terrorism|volume=14|issue=3|jstor=26918296s|issn=2334-3745}}
* {{cite book|last=Nomad|first=Max|contribution=The Anarchist Tradition|editor1-last=Drachkovitch|editor1-first=Milorad M.|title=Revolutionary Internationals 1864–1943|publisher=Stanford University Press|page=88|year=1966|isbn=978-0-8047-0293-5}}
* {{cite journal|last=Osgood|first=Herbert L.|title=Scientific Anarchism|publisher=The Academy of Political Science|journal=Political Science Quarterly|volume=4|issue=1|date=March 1889|pages=1–36|jstor=2139424|doi=10.2307/2139424}}
* {{cite book|last=Otero|first=Carlos Peregrín|year=1994|title=Noam Chomsky: Critical Assessments|volume=2–3|location=London|publisher=Routledge|isbn=978-0-415010-05-4}}
* {{cite book|last=Parry|first=Richard|title=The Bonnot Gang|url=https://archive.org/details/bonnotgang0000parr|url-access=registration|year=1987|publisher=Rebel Press|isbn=978-0-946061-04-4}}
* {{cite book|last=Pernicone|first=Nunzio|title=Italian Anarchism, 1864–1892|url=https://books.google.com/books?id=3ttgjwEACAAJ|year=2009|publisher=Princeton University Press|isbn=978-0-691-63268-1}}
* {{cite book|last=Pierson|first=Christopher|title=Just Property: Enlightenment, Revolution, and History|url=https://books.google.com/books?id=7jvKDAAAQBAJ&pg=PA187|year=2013|publisher=Oxford University Press|isbn=978-0-19-967329-2}}
* {{cite book|last=Ramnath|first=Maia|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|year=2019|publisher=Springer|isbn=978-3-319-75620-2|chapter=Non-Western Anarchisms and Postcolonialism}}
* {{cite book|last=Robé|first=Chris|title=Breaking the Spell: A History of Anarchist Filmmakers, Videotape Guerrillas, and Digital Ninjas|url=https://www.researchgate.net/publication/336710855|year=2017|publisher=PM Press|isbn=978-1-629-63233-9}}
* {{cite book|last=Rogers|first=Tristan J.|year=2020|title=The Authority of Virtue: Institutions and Character in the Good Society|location=London|publisher=Routledge|isbn=978-1-000-22264-7}}
* {{cite book|last=Rupert|first=Mark|title=Globalization and International Political Economy|publisher=Rowman & Littlefield Publishers|location=Lanham|year=2006|isbn=978-0-7425-2943-4|url=https://archive.org/details/globalizationint00rupe}}
* {{cite book|last=Ryley|first=Peter|chapter=Individualism|pages=225–236|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Shannon|first=Deric|chapter=Anti-Capitalism and Libertarian Political Economy|pages=91–106|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Skirda|first=Alexandre|title=Facing the Enemy: A History of Anarchist Organization From Proudhon to May 1968|publisher=AK Press|year=2002|isbn=978-1-902593-19-7}}
* {{cite book|last=Sylvan|year=2007|first=Richard|section=Anarchism|editor=Robert E. Goodin|editor2=Philip Pettit|editor3=Thomas Pogge|title=A Companion to Contemporary Political Philosophy|edition=2nd|series=Blackwell Companions to Philosophy|volume=5|publisher=Blackwell Publishing|isbn=978-1-4051-3653-2|author-link=Richard Sylvan|editor1-link=Robert E. Goodin|editor2-link=Philip Pettit|editor3-link=Thomas Pogge}}
* {{cite book|last=Suissa|first=Judith|chapter=Anarchist Education|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Thomas|first=Paul|title=Karl Marx and the Anarchists|publisher=Routledge & Kegan Paul|location=London|year=1985|isbn=978-0-7102-0685-5}}
* {{cite book|editor-last=Tucker|editor-first1=Robert C.|year=1978|title=The Marx-Engels Reader|edition=2nd|location=New York|publisher=W. W. Norton & Company|oclc=3415145|isbn=0-393-05684-8}}
* {{cite book|last=Turcato|first=Davide|chapter=Anarchist Communism|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Van der Walt|first=Lucien|chapter=Syndicalism|pages=249–264|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last=Ward|first=Colin|date=1973|title=The Role of the State|journal=Education Without Schools|pages=39–48}}
* {{cite book|last=Ward|first=Colin|author-link=Colin Ward|year=2004|url=https://books.google.com/books?id=nkgSDAAAQBAJ|title=Anarchism: A Very Short Introduction|location=Oxford|publisher=Oxford University Press|isbn=978-0-19-280477-8}}
* {{cite book|last=Walter|first=Nicholas|year=2002|title=About Anarchism|location=London|publisher=Freedom Press|isbn=978-0-900384-90-5}}
* {{cite journal|last=Wendt|first=Fabian|year=2020|title=Against Philosophical Anarchism|journal=Law and Philosophy|volume=39|issue=5|pages=527–544|doi=10.1007/s10982-020-09377-4|s2cid=213742949}}
* {{cite book|last=Wilbur|first=Shawn|chapter=Mutualism|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=Springer Publishing|isbn=978-3-319-75620-2}}
* {{cite journal|last=Williams|first=Dana M.|year=2015|title=Black Panther Radical Factionalization and the Development of Black Anarchism|journal=Journal of Black Studies|location=Thousand Oaks|publisher=SAGE Publishing|volume=46|issue=7|pages=678–703|doi=10.1177/0021934715593053|jstor=24572914|s2cid=145663405}}
* {{cite journal|last=Williams|first=Dana M.|author-mask=2|title=Contemporary Anarchist and Anarchistic Movements|journal=Sociology Compass|publisher=Wiley|volume=12|issue=6|pages=e12582|year=2018|issn=1751-9020|doi=10.1111/soc4.12582}}
* {{cite book|last=Williams|first=Dana M.|author-mask=2|chapter=Tactics: Conceptions of Social Change, Revolution, and Anarchist Organisation|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=Springer Publishing|isbn=978-3-319-75620-2}}
* {{Cite journal |last1=Williams |first1=Leonard |title=Anarchism Revived |journal=New Political Science |volume=29 |issue=3 |pages=297–312 |date=September 2007 |language=en |doi=10.1080/07393140701510160 |issn=0739-3148 }}
* {{cite journal|last=Williams|first=Leonard|author-mask=2|year=2010|title=Hakim Bey and Ontological Anarchism|journal=Journal for the Study of Radicalism|location=East Lansing|publisher=Michigan State University Press|volume=4|issue=2|pages=109–137|doi=10.1353/jsr.2010.0009|jstor=41887660|s2cid=143304524}}
{{refend}}

==== Tertiary sources ====
{{refbegin|35em|indent=yes}}
* {{cite web|last=Coutinho|first=Steve|title=Zhuangzi|publisher=Internet Encyclopedia of Philosophy|date=3 March 2016|url=http://www.iep.utm.edu/zhuangzi/|archive-url=https://web.archive.org/web/20160303175106/http://www.iep.utm.edu/zhuangzi/|archive-date=3 March 2016|url-status=live|access-date=5 March 2019}}
* {{cite book|last=De George|first=Richard T.|editor=Ted Honderich|editor-link=Ted Honderich|title=The Oxford Companion to Philosophy|publisher=[[Oxford University Press]]|isbn=978-0-19-926479-7|year=2005}}
* {{cite encyclopedia|last=Fiala|first=Andrew|title=Anarchism|encyclopedia=[[Stanford Encyclopedia of Philosophy]]|year=2017|url=https://plato.stanford.edu/entries/anarchism/|publisher=Metaphysics Research Lab, Stanford University}}
* {{cite encyclopedia|last1=Miller|first1=Martin A.|last2=Dirlik|first2=Arif|last3=Rosemont|first3=Franklin|last4=Augustyn|first4=Adam|last5=Duignan|first5=Brian|last6=Lotha|first6=Gloria|title=Anarchism In The Arts|encyclopedia=[[Encyclopaedia Britannica]]|year=2019|url=https://www.britannica.com/topic/anarchism/Anarchism-in-the-arts#info-article-contributors}}
* {{cite encyclopedia|last1=Morris|first1=Andrew|editor-last1=Hamowy|editor-first1=Ronald|editor-link=Ronald Hamowy|title=Anarcho-capitalism|pages=13–14|year=2008|url=https://books.google.com/books?id=yxNgXs3TkJYC&pg=PT51|isbn=978-1-4129-6580-4|publisher=SAGE; Cato Institute|encyclopedia=The Encyclopedia of Libertarianism|oclc=191924853|doi=10.4135/9781412965811.n8}}
* {{cite book|last1=McLean|first1=Iain|first2=Alistair|last2=McMillan|title=The Concise Oxford Dictionary of Politics|url=https://archive.org/details/oxfordconcisedic00iain|url-access=registration|year=2003|publisher=Oxford University Press|isbn=978-0-19-280276-7}}
* {{cite web|title=Definition of Anarchism|work=Merriam-Webster|year=2019|url=https://www.merriam-webster.com/dictionary/anarchism|ref={{sfnref|Merriam-Webster|2019}}|access-date=28 February 2019}}
* {{cite book|last=Miller|first=David|title=The Blackwell Encyclopaedia of Political Thought|url=https://books.google.com/books?id=NIZfQTd3nSMC|date=26 August 1991|publisher=Wiley|isbn=978-0-631-17944-3}}
* {{cite encyclopedia|last=Ostergaard|first=Geoffrey|author-link=Geoffrey Ostergaard|year=2003|url=https://books.google.com/books?id=JJmdpqJwkwwC&pg=PA12|title=Anarchism|editor-last=Outhwaite|editor-first=William|encyclopedia=The Blackwell Dictionary of Modern Social Thought|edition=2nd|location=Malden, Massachusetts|publisher=Blackwell Publishing|isbn=978-0-631-22164-7|oclc=49704935}}
* {{cite book|chapter=Anarchy|title=Oxford English Dictionary|edition=3rd|publisher=Oxford University Press|date=September 2005|ref={{harvid|Oxford English Dictionary|2005}}}}
{{refend}}

== Further reading ==
* {{cite book|last=Barclay|first=Harold B.|author-link=Harold Barclay|title=People Without Government: An Anthropology of Anarchy|url=https://books.google.com/books?id=MrFHQgAACAAJ|year=1990|publisher=Kahn & Averill|isbn=978-0-939306-09-1}}
* {{cite book|last=Edmundson|first=William A.|title=Three Anarchical Fallacies: An Essay on Political Authority|url=https://books.google.com/books?id=q_gClKUbJyYC|date=2007|publisher=Cambridge University Press|isbn=978-0-521-03751-8}} Criticism of philosophical anarchism.
* {{cite book|last=Harper|first=Clifford|author-link=Clifford Harper|year=1987|url=https://books.google.com/books?id=W63aAAAAMAAJ|title=Anarchy: A Graphic Guide|publisher=Camden Press|isbn=978-0-948491-22-1}}
* {{cite book|last=Huemer|first=Michael|year=2012|title=The Problem of Political Authority: An Examination of the Right to Coerce and the Duty to Obey|location=London|publisher=Palgrave Macmillan|isbn=978-1-137-28164-7}} A defence of philosophical anarchism, stating that "both kinds of 'anarchism' [i.e. philosophical and political anarchism] are philosophical and political claims." (p.&nbsp;137)
* {{cite book|last=Le Guin|first=Ursula K.|author-link=Ursula K. Le Guin|title=The Dispossessed|year=2009|publisher=HarperCollins|title-link=The Dispossessed}} Anarchistic popular fiction novel.<!-- Gifford 2019, p 580. -->
* {{cite book|last=Kinna|first=Ruth|author-link=Ruth Kinna|title=Anarchism: A Beginners Guide|url=https://books.google.com/books?id=LLLaAAAAMAAJ|year=2005|publisher=Oneworld|isbn=978-1-85168-370-3}}
* {{cite book|last=Sartwell|first=Crispin|author-link=Crispin Sartwell|title=Against the State: An Introduction to Anarchist Political Theory|publisher=SUNY Press|year=2008|isbn=978-0-7914-7447-1|url=https://books.google.com/books?id=bk-aaMVGKO0C}}
* {{cite book|last=Scott|first=James C.|author-link=James C. Scott|year=2012|title=Two Cheers for Anarchism: Six Easy Pieces on Autonomy, Dignity, and Meaningful Work and Play|title-link=Two Cheers for Anarchism|location=Princeton, New Jersey|publisher=Princeton University Press|isbn=978-0-691-15529-6}}
* {{cite book|last=Wolff|first=Robert Paul|author-link=Robert Paul Wolff|year=1998|title=In Defense of Anarchism|title-link=In Defense of Anarchism|publisher=University of California Press|isbn=978-0-520-21573-3}} An argument for philosophical anarchism.
* {{cite journal|last=Woodcock|first=George|author-link=George Woodcock|date=January 1962|url=https://www.historytoday.com/archive/anarchism-spain|title=Anarchism in Spain|journal=History Today|volume=12|issue=1|pages=22–32|access-date=14 October 2020}}

== External links ==
* [http://dwardmac.pitzer.edu/ Anarchy Archives]. [[Anarchy Archives]] is an online research center on the history and theory of anarchism.
{{subject bar|portal1=Anarchism|wikt=yes|commons=yes|q=yes|s=yes|b=yes|d=yes|v=yes|n=yes}}

<!-- Attention! The external link portion of this article regularly grows far beyond manageable size. PLEASE only list an outside link if it applies to anarchism in general, is somewhat noteworthy, and has consensus on the talkpage. Links to sites which cover anarchist submovements will be routinely moved to subarticles to keep this article free of clutter. -->
{{prone to spam|date=November 2014}}
<!-- {{no more links}}. Please be cautious adding more external links. Wikipedia is not a collection of links and should not be used for advertising. Excessive or inappropriate links will be removed. See [[Wikipedia:External links]] and [[Wikipedia:Spam]] for details. If there are already suitable links, propose additions or replacements on the article's talk page, or submit your link to the relevant category at Curlie (curlie.org) – and link there using {{curlie}}. -->

{{-}}
{{anarchism}}
{{anarchies}}
{{libertarian socialism}}
{{libertarianism}}
{{philosophy topics}}
{{political culture}}
{{political ideologies}}
{{social and political philosophy}}
{{authority control}}

[[Category:Anarchism| ]]
[[Category:Anti-capitalism]]
[[Category:Anti-fascism]]
[[Category:Economic ideologies]]
[[Category:Left-wing politics]]
[[Category:Libertarian socialism]]
[[Category:Libertarianism]]
[[Category:Political culture]]
[[Category:Political movements]]
[[Category:Political ideologies]]
[[Category:Social theories]]
[[Category:Socialism]]
"""

In [19]:
assert getArticleFeaturesRegex(wt) == getArticleFeaturesMWP(wt), f'{getArticleFeaturesRegex(wt)} vs. {getArticleFeaturesMWP(wt)}'


In [20]:
assert getArticleFeaturesRegex(long_wt) == getArticleFeaturesMWP(long_wt), f'{getArticleFeaturesRegex(long_wt)} vs. {getArticleFeaturesMWP(long_wt)}'

AssertionError: [(97943, 1, 437, 383, 12, 12, 28)] vs. [(97943, 1, 444, 386, 12, 12, 28)]

In [21]:
%timeit getArticleFeaturesRegex(wt)

174 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
%timeit getArticleFeaturesMWP(wt)

2.38 ms ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit getArticleFeaturesRegex(long_wt)

6.75 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit getArticleFeaturesMWP(long_wt)

143 ms ± 832 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Data to help in setting min thresholds

In [27]:
# Thresholds after the going through the preprocessing
print_for_hive = False
do_execute = True
pctile = 0.95

# PERCENTILE_APPROX(num_refs / page_len, {pctile}) AS max_refs_per_len,

query = f"""
SELECT
  wiki_db,
  COUNT(1) AS num_pages,
  PERCENTILE(page_length, {pctile}) AS max_length,
  PERCENTILE(num_media, {pctile}) AS max_media,
  PERCENTILE(num_categories, {pctile}) AS max_cats,
  PERCENTILE_APPROX(num_refs / SQRT(page_length), {pctile}) AS max_refs,
  PERCENTILE_APPROX(num_headings / SQRT(page_length), {pctile}) AS max_headings,
  PERCENTILE_APPROX(SQRT(num_wikilinks) / SQRT(page_length), {pctile}) AS max_links
FROM {allwikis_tablename}
GROUP BY
  wiki_db
ORDER BY
  num_pages DESC
"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    spark.sql(query).show(500, False)


SELECT
  wiki_db,
  COUNT(1) AS num_pages,
  PERCENTILE(page_length, 0.95) AS max_length,
  PERCENTILE(num_media, 0.95) AS max_media,
  PERCENTILE(num_categories, 0.95) AS max_cats,
  PERCENTILE_APPROX(num_refs / SQRT(page_length), 0.95) AS max_refs,
  PERCENTILE_APPROX(num_headings / SQRT(page_length), 0.95) AS max_headings,
  PERCENTILE_APPROX(SQRT(num_wikilinks) / SQRT(page_length), 0.95) AS max_links
FROM isaacj.qual_features_regex_2021_12_wikitext
GROUP BY
  wiki_db
ORDER BY
  num_pages DESC

+----------------+---------+------------------+---------+-----------------+--------------------+--------------------+-------------------+
|wiki_db         |num_pages|max_length        |max_media|max_cats         |max_refs            |max_headings        |max_links          |
+----------------+---------+------------------+---------+-----------------+--------------------+--------------------+-------------------+
|enwiki          |6431489  |27441.599999999627|3.0      |14.0             |0.29435

In [23]:
# Thresholds if there was no pre-processing
print_for_hive = False
do_execute = True
pctile = 0.95

# PERCENTILE_APPROX(num_refs / page_len, {pctile}) AS max_refs_per_len,

query = f"""
SELECT
  wiki_db,
  COUNT(1) AS num_pages,
  PERCENTILE(page_length, {pctile}) AS max_length,
  PERCENTILE(num_media, {pctile}) AS max_media,
  PERCENTILE(num_refs, {pctile}) AS max_refs,
  PERCENTILE(num_headings, {pctile}) AS max_headings,
  PERCENTILE(num_categories, {pctile}) AS max_cats,
  PERCENTILE(num_wikilinks, {pctile}) AS max_links
FROM {allwikis_tablename}
GROUP BY
  wiki_db
ORDER BY
  num_pages DESC
"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    spark.sql(query).show(500, False)


SELECT
  wiki_db,
  COUNT(1) AS num_pages,
  PERCENTILE(page_length, 0.95) AS max_length,
  PERCENTILE(num_media, 0.95) AS max_media,
  PERCENTILE(num_refs, 0.95) AS max_refs,
  PERCENTILE(num_headings, 0.95) AS max_headings,
  PERCENTILE(num_categories, 0.95) AS max_cats,
  PERCENTILE(num_wikilinks, 0.95) AS max_links
FROM isaacj.qual_features_regex_2021_12_wikitext
GROUP BY
  wiki_db
ORDER BY
  num_pages DESC

+----------------+---------+------------------+---------+-----------------+------------------+-----------------+------------------+
|wiki_db         |num_pages|max_length        |max_media|max_refs         |max_headings      |max_cats         |max_links         |
+----------------+---------+------------------+---------+-----------------+------------------+-----------------+------------------+
|enwiki          |6431489  |27441.599999999627|3.0      |39.0             |14.0              |14.0             |146.0             |
|cebwiki         |6098451  |9566.0            |0.0     